# 🛰️ GeoVeil CN0 Analysis Widget

**Interactive GNSS C/N₀ Analysis powered by the Rust `geoveil-cn0` library**

- **90% Rust** - All parsing, analysis, anomaly detection in compiled Rust
- **10% Python** - Only UI widgets and Plotly visualization

## 1. Build Rust Library

In [ ]:
# === LINUX/MAC BUILD ===
import subprocess
import sys
import os
import platform
import glob

print("🐧 Linux/Mac Build for geoveil_cn0")
print("=" * 60)

if platform.system() == 'Windows':
    print("⚠️ This cell is for Linux/Mac. Use the Windows build cell instead.")
else:
    NOTEBOOK_DIR = os.getcwd()
    
    # Setup PATH for cargo/maturin
    cargo_bin = os.path.expanduser("~/.cargo/bin")
    local_bin = os.path.expanduser("~/.local/bin")
    os.environ["PATH"] = f"{cargo_bin}:{local_bin}:" + os.environ.get("PATH", "")
    
    # Find or extract library
    LIB_PATH = None
    for candidate in [
        os.path.join(NOTEBOOK_DIR, 'geoveil-cn0'),
        os.path.join(NOTEBOOK_DIR, 'geoveil_cn0'),
    ]:
        if os.path.exists(os.path.join(candidate, 'Cargo.toml')):
            LIB_PATH = candidate
            break
    
    if not LIB_PATH:
        tar_file = os.path.join(NOTEBOOK_DIR, 'geoveil-cn0.tar.gz')
        if os.path.exists(tar_file):
            print("📦 Extracting geoveil-cn0.tar.gz...")
            import tarfile
            with tarfile.open(tar_file, 'r:gz') as tar:
                tar.extractall(NOTEBOOK_DIR)
            LIB_PATH = os.path.join(NOTEBOOK_DIR, 'geoveil-cn0')
    
    if not LIB_PATH:
        raise FileNotFoundError("geoveil-cn0 directory not found")
    
    print(f"📁 Library: {LIB_PATH}")
    print(f"🐍 Python: {sys.version.split()[0]}")
    
    # Check Rust
    print("\n🔧 Checking Rust...")
    result = subprocess.run(['rustc', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ {result.stdout.strip()}")
    else:
        print("❌ Rust not found. Install: curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh")
        raise RuntimeError("Rust not installed")
    
    # Check maturin
    print("\n📦 Checking maturin...")
    result = subprocess.run([sys.executable, '-m', 'maturin', '--version'], capture_output=True, text=True)
    if result.returncode != 0:
        print("   Installing maturin...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'maturin>=1.4'])
    result = subprocess.run([sys.executable, '-m', 'maturin', '--version'], capture_output=True, text=True)
    print(f"✅ {result.stdout.strip()}")
    
    # Build
    print("\n🔨 Building (this may take 2-5 minutes)...")
    print("-" * 60)
    
    wheel_dir = os.path.join(LIB_PATH, 'target', 'wheels')
    
    # Set environment for Python 3.13+ compatibility
    env = os.environ.copy()
    env['PYO3_USE_ABI3_FORWARD_COMPATIBILITY'] = '1'
    
    result = subprocess.run(
        [sys.executable, '-m', 'maturin', 'build', '--release'],
        cwd=LIB_PATH,
        env=env,
        capture_output=True,
        text=True
    )
    
    print(result.stdout[-2000:] if result.stdout else "")
    
    if result.returncode != 0:
        print(f"❌ Build failed:\n{result.stderr[-1500:]}")
        raise RuntimeError("Build failed")
    
    print("-" * 60)
    
    # Find and install wheel
    wheels = glob.glob(os.path.join(wheel_dir, 'geoveil_cn0*.whl'))
    
    if wheels:
        wheel = max(wheels, key=os.path.getctime)
        print(f"\n📥 Installing {os.path.basename(wheel)}...")
        
        result = subprocess.run([
            sys.executable, '-m', 'pip', 'install',
            '--force-reinstall', '--no-deps', '-q', wheel
        ], capture_output=True, text=True)
        
        if result.returncode == 0:
            print("✅ Installed!")
        else:
            print(f"❌ Install failed: {result.stderr}")
            raise RuntimeError("Install failed")
    else:
        print("❌ No wheel found")
        raise RuntimeError("Build failed - no wheel produced")
    
    # Test import
    print("\n🧪 Testing import...")
    if 'geoveil_cn0' in sys.modules:
        del sys.modules['geoveil_cn0']
    
    import geoveil_cn0 as gcn0
    print(f"✅ geoveil_cn0 v{gcn0.VERSION} loaded!")


## 1.1 Build Rust Library on WIN

In [ ]:
# === WINDOWS BUILD with Python 3.13 Compatibility ===
import subprocess
import sys
import os
import platform
import glob

print("🪟 Windows Build for geoveil_cn0 (Python 3.13 compat)")
print("=" * 60)

if platform.system() != 'Windows':
    print("⚠️ This cell is for Windows only")
else:
    NOTEBOOK_DIR = os.getcwd()
    
    # Find library
    LIB_PATH = None
    for candidate in [
        os.path.join(NOTEBOOK_DIR, 'geoveil-cn0'),
        os.path.join(NOTEBOOK_DIR, 'geoveil_cn0'),
    ]:
        if os.path.exists(os.path.join(candidate, 'Cargo.toml')):
            LIB_PATH = candidate
            break
    
    if not LIB_PATH:
        tar_file = os.path.join(NOTEBOOK_DIR, 'geoveil-cn0.tar.gz')
        if os.path.exists(tar_file):
            import tarfile
            with tarfile.open(tar_file, 'r:gz') as tar:
                tar.extractall(NOTEBOOK_DIR)
            LIB_PATH = os.path.join(NOTEBOOK_DIR, 'geoveil-cn0')
    
    if not LIB_PATH:
        raise FileNotFoundError("geoveil-cn0 directory not found")
    
    print(f"📁 Library: {LIB_PATH}")
    print(f"🐍 Python: {sys.version}")
    
    # Find vcvars64.bat
    VS_PATHS = [
        r"C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Auxiliary\Build\vcvars64.bat",
        r"C:\Program Files\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\Build\vcvars64.bat",
        r"C:\Program Files\Microsoft Visual Studio\2022\Professional\VC\Auxiliary\Build\vcvars64.bat",
    ]
    
    VCVARS = None
    for path in VS_PATHS:
        if os.path.exists(path):
            VCVARS = path
            break
    
    if not VCVARS:
        raise FileNotFoundError("VS Build Tools not found")
    
    print(f"🔧 VS Tools: Found")
    
    # Create build script with PYO3_USE_ABI3_FORWARD_COMPATIBILITY
    build_script = os.path.join(NOTEBOOK_DIR, '_build_geoveil.bat')
    wheel_dir = os.path.join(LIB_PATH, 'target', 'wheels')
    
    script_content = f'''@echo off
call "{VCVARS}" >nul 2>&1
cd /d "{LIB_PATH}"
set PYO3_USE_ABI3_FORWARD_COMPATIBILITY=1
"{sys.executable}" -m maturin build --release
'''
    
    with open(build_script, 'w') as f:
        f.write(script_content)
    
    print(f"\n🔨 Building with ABI3 forward compatibility...")
    print("-" * 60)
    
    result = subprocess.run(['cmd', '/c', build_script], cwd=LIB_PATH)
    
    print("-" * 60)
    
    try:
        os.remove(build_script)
    except:
        pass
    
    # Find and install wheel
    wheels = glob.glob(os.path.join(wheel_dir, 'geoveil_cn0*.whl'))
    
    if wheels:
        wheel = max(wheels, key=os.path.getctime)
        print(f"\n📥 Installing {os.path.basename(wheel)}...")
        
        result = subprocess.run([
            sys.executable, '-m', 'pip', 'install', 
            '--force-reinstall', '--no-deps', wheel
        ], capture_output=True, text=True)
        
        if result.returncode == 0:
            print("✅ Installed!")
        else:
            print(f"❌ Install failed: {result.stderr}")
            raise RuntimeError("Install failed")
    else:
        print("❌ No wheel found")
        raise RuntimeError("Build failed")
    
    # Test import
    print("\n🧪 Testing import...")
    if 'geoveil_cn0' in sys.modules:
        del sys.modules['geoveil_cn0']
    
    import geoveil_cn0 as gcn0
    print(f"✅ geoveil_cn0 v{gcn0.VERSION} loaded!")

## Install library from Pypi

In [ ]:
#!pip install geoveil-cn0

### Import

In [ ]:
import geoveil_cn0 as gcn0
print(f'✅ geoveil_cn0 v{gcn0.VERSION}')

## 2. Dependencies

In [ ]:
!pip install plotly pandas numpy ipywidgets -q

In [ ]:
import os, json, tempfile, base64
from pathlib import Path
from datetime import datetime
from collections import defaultdict
import numpy as np, pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import geoveil_cn0 as gcn0
print('✅ Imports ready')

## 3. Visualization Functions

All data comes from Rust: `result.get_skyplot_data()`, `result.get_timeseries_data()`, etc.

In [ ]:
COLORS = {'GPS':'#3b82f6','Galileo':'#22c55e','GLONASS':'#ef4444','BeiDou':'#f59e0b','QZSS':'#8b5cf6','IRNSS':'#06b6d4'}
RATING = {'excellent':'#22c55e','good':'#84cc16','fair':'#eab308','poor':'#f97316','critical':'#ef4444'}
SEVERITY = {'Critical':'#ef4444','High':'#f97316','Medium':'#eab308','Low':'#22c55e'}
def get_const(sv): return {'G':'GPS','E':'Galileo','R':'GLONASS','C':'BeiDou','J':'QZSS','I':'IRNSS'}.get(sv[0] if sv else '','Other')

In [ ]:
def create_skyplot(r):
    data = r.get_skyplot_data()
    traces = data.get('traces', [])
    if not traces: return go.Figure().add_annotation(text='No position data',xref='paper',yref='paper',x=0.5,y=0.5,showarrow=False)
    fig = go.Figure()
    for i,tr in enumerate(traces):
        fig.add_trace(go.Scatterpolar(r=tr['r'],theta=tr['theta'],mode='markers',name=tr['constellation'],
            marker=dict(size=9,color=tr['cn0'],colorscale='Viridis',cmin=25,cmax=55,colorbar=dict(title='CN0') if i==0 else None,showscale=(i==0)),
            text=tr.get('text',[]),hoverinfo='text'))
    cov = data.get('coverage',{}).get('coverage_percent',0)
    fig.update_layout(title=f'CN0 Skyplot ({cov:.1f}%)',width=620,height=620,
        polar=dict(radialaxis=dict(range=[0,90],tickvals=[0,30,60,90],ticktext=['90°','60°','30°','0°']),
                  angularaxis=dict(tickvals=[0,45,90,135,180,225,270,315],ticktext=['N','NE','E','SE','S','SW','W','NW'],direction='clockwise',rotation=90)))
    return fig

In [ ]:
def create_timeseries(r):
    d = r.get_timeseries_data()
    ts = pd.to_datetime(d.get('timestamps',[]))
    if len(ts)==0: return go.Figure().add_annotation(text='No data',xref='paper',yref='paper',x=0.5,y=0.5,showarrow=False)
    fig = make_subplots(rows=2,cols=1,shared_xaxes=True,vertical_spacing=0.08,row_heights=[0.7,0.3])
    fig.add_trace(go.Scatter(x=ts,y=d.get('cn0_mean',[]),name='Overall',line=dict(color='black',width=2.5)),row=1,col=1)
    for c,cd in d.get('by_constellation',{}).items():
        fig.add_trace(go.Scatter(x=pd.to_datetime(cd['timestamps']),y=cd['cn0'],name=c,line=dict(color=COLORS.get(c,'#888'))),row=1,col=1)
    fig.add_trace(go.Bar(x=ts,y=d.get('satellite_counts',[]),marker_color='#6b7280',showlegend=False),row=2,col=1)
    fig.add_hline(y=35,line_dash='dash',line_color='orange',row=1,col=1)
    fig.update_layout(height=500,title='CN0 Timeseries',legend=dict(orientation='h',y=1.1))
    fig.update_yaxes(title_text='CN0 (dB-Hz)',row=1,col=1)
    return fig

In [ ]:
def create_heatmap(r):
    hm = r.get_skyplot_data().get('heatmap',{})
    if not hm.get('cn0_grid'): return go.Figure().add_annotation(text='No heatmap',xref='paper',yref='paper',x=0.5,y=0.5,showarrow=False)
    fig = go.Figure(go.Heatmap(z=hm['cn0_grid'],x=hm['azimuth_bins'],y=hm['elevation_bins'],colorscale='Viridis',colorbar=dict(title='CN0')))
    fig.update_layout(title='CN0 Heatmap',xaxis_title='Azimuth (°)',yaxis_title='Elevation (°)',width=700,height=450)
    return fig

In [ ]:
def create_elevation_plot(r):
    traces = r.get_skyplot_data().get('traces',[])
    if not traces: return go.Figure().add_annotation(text='No data',xref='paper',yref='paper',x=0.5,y=0.5,showarrow=False)
    fig = go.Figure()
    for tr in traces:
        elevs = [90-x for x in tr.get('r',[])]
        fig.add_trace(go.Scatter(x=elevs,y=tr.get('cn0',[]),mode='markers',name=tr['constellation'],marker=dict(color=COLORS.get(tr['constellation'],'#888'),size=5,opacity=0.5)))
    elev = np.linspace(5,90,50)
    fig.add_trace(go.Scatter(x=elev,y=35+15*(elev/90),mode='lines',name='Expected',line=dict(color='black',dash='dash')))
    fig.add_hline(y=35,line_dash='dot',line_color='orange')
    fig.update_layout(title='CN0 vs Elevation',xaxis_title='Elevation (°)',yaxis_title='CN0 (dB-Hz)',width=700,height=450)
    return fig

In [ ]:
def create_satellite_bars(r):
    stats = r.get_satellite_stats()
    if not stats: return go.Figure().add_annotation(text='No data',xref='paper',yref='paper',x=0.5,y=0.5,showarrow=False)
    by_c = defaultdict(list)
    for sv,s in stats.items(): by_c[get_const(sv)].append((sv,s['mean'],s['std_dev']))
    fig = go.Figure()
    for c in sorted(by_c):
        d = sorted(by_c[c])
        fig.add_trace(go.Bar(x=[x[0] for x in d],y=[x[1] for x in d],name=c,marker_color=COLORS.get(c,'#888'),error_y=dict(type='data',array=[x[2] for x in d],visible=True)))
    fig.add_hline(y=40,line_dash='dash',line_color='green')
    fig.update_layout(title='CN0 by Satellite',barmode='group',height=400,legend=dict(orientation='h',y=1.1))
    return fig

In [ ]:
def create_constellation_pie(r):
    stats = r.get_satellite_stats()
    if not stats: return go.Figure()
    by_c = defaultdict(int)
    for sv,s in stats.items(): by_c[get_const(sv)] += s['count']
    fig = go.Figure(go.Pie(labels=list(by_c.keys()),values=list(by_c.values()),marker_colors=[COLORS.get(c,'#888') for c in by_c],hole=0.4,textinfo='label+percent'))
    fig.update_layout(title=f'Observations ({sum(by_c.values()):,})',height=380)
    return fig

In [ ]:
def create_quality_gauge(r):
    fig = go.Figure(go.Indicator(mode='gauge+number',value=r.score,title={'text':f'Quality: {r.rating.upper()}'},
        gauge={'axis':{'range':[0,100]},'bar':{'color':RATING.get(r.rating,'#888')},
               'steps':[{'range':[0,30],'color':'#fee2e2'},{'range':[30,50],'color':'#fef3c7'},{'range':[50,70],'color':'#fef9c3'},{'range':[70,85],'color':'#dcfce7'},{'range':[85,100],'color':'#bbf7d0'}],
               'threshold':{'line':{'color':'black','width':3},'thickness':0.8,'value':70}}))
    fig.update_layout(height=300,width=380)
    return fig

In [ ]:
def create_quality_radar(r):
    q = r.quality_score
    cats = ['CN0','Availability','Continuity','Stability','Diversity','CN0']
    vals = [q.cn0_quality,q.availability,q.continuity,q.stability,q.diversity,q.cn0_quality]
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(r=vals,theta=cats,fill='toself',fillcolor='rgba(59,130,246,0.3)',line=dict(color='#3b82f6',width=2),name='Quality'))
    fig.add_trace(go.Scatterpolar(r=[70]*6,theta=cats,fill='none',line=dict(color='green',dash='dash'),name='Good'))
    fig.update_layout(title='Quality Breakdown',polar=dict(radialaxis=dict(range=[0,100])),height=400,width=420)
    return fig

In [ ]:
# Create_anomaly_timeline function

def create_anomaly_timeline(r):
    """Create anomaly timeline plot from Rust analysis result"""
    anomalies = r.get_anomalies()
    
    if not anomalies:
        fig = go.Figure()
        fig.add_annotation(
            text='✅ No anomalies detected',
            xref='paper', yref='paper',
            x=0.5, y=0.5,
            showarrow=False,
            font=dict(size=18, color='green')
        )
        fig.update_layout(title='Anomaly Timeline', height=300)
        return fig
    
    # Parse data - handle missing keys gracefully
    parsed_data = []
    for a in anomalies:
        try:
            # Get timestamp - try multiple keys
            ts_str = a.get('start_time') or a.get('timestamp') or ''
            if ts_str:
                ts = pd.to_datetime(ts_str)
            else:
                continue  # Skip if no timestamp
            
            parsed_data.append({
                'time': ts,
                'type': a.get('anomaly_type', a.get('type', 'Unknown')),
                'severity': a.get('severity', 'Low'),
                'cn0_drop': float(a.get('cn0_drop', a.get('cn0_drop_db', 0)) or 0),
                'description': a.get('description', ''),
                'satellites': a.get('affected_satellites', ''),
            })
        except Exception as e:
            continue  # Skip malformed anomalies
    
    if not parsed_data:
        fig = go.Figure()
        fig.add_annotation(
            text='⚠️ Anomalies found but timestamps invalid',
            xref='paper', yref='paper',
            x=0.5, y=0.5,
            showarrow=False,
            font=dict(size=14, color='orange')
        )
        fig.update_layout(title='Anomaly Timeline', height=300)
        return fig
    
    # Create figure
    fig = go.Figure()
    
    # Plot by severity
    severity_colors = {
        'Critical': '#ef4444',  # Red
        'High': '#f97316',      # Orange  
        'Medium': '#eab308',    # Yellow
        'Low': '#22c55e',       # Green
        'critical': '#ef4444',
        'high': '#f97316',
        'medium': '#eab308',
        'low': '#22c55e',
    }
    
    for severity in ['Critical', 'High', 'Medium', 'Low', 'critical', 'high', 'medium', 'low']:
        sev_data = [d for d in parsed_data if d['severity'] == severity]
        if sev_data:
            # Normalize severity name for display
            display_name = severity.capitalize()
            
            fig.add_trace(go.Scatter(
                x=[d['time'] for d in sev_data],
                y=[d['cn0_drop'] for d in sev_data],
                mode='markers',
                marker=dict(
                    size=10,
                    color=severity_colors.get(severity, '#888'),
                    opacity=0.7
                ),
                name=f"{display_name} ({len(sev_data)})",
                text=[f"{d['type']}<br>{d['description']}" for d in sev_data],
                hovertemplate="Time: %{x}<br>CN0 Drop: %{y:.1f} dB<br>%{text}<extra></extra>"
            ))
    
    fig.update_layout(
        title=f'Anomaly Timeline ({len(parsed_data)} events)',
        xaxis_title='Time (UTC)',
        xaxis=dict(type='date'),  # Force date axis
        yaxis_title='CN0 Drop (dB)',
        height=400,
        legend=dict(orientation='h', y=1.1)
    )
    
    return fig

print('✅ Fixed create_anomaly_timeline function')

## 4. Interactive Widget

Complete widget with 5 presets and 9 visualization tabs.

In [3]:
# CN0 Analysis Widget with Presets, Lock Integrity & Fixed Anomaly Graph
# =========================================================================
# Features:
# - Research-based preset configurations (ITU, Stanford GPS Lab, GPS Solutions)
# - Lock Integrity metric (cycle slips + data gaps)
# - Fixed anomaly graph using pd.to_datetime for robust timestamp parsing
# - Quick mode for fast analysis
# - File path support + auto-download navigation
# =========================================================================

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
import gzip
import tempfile
from pathlib import Path
from datetime import datetime, timedelta
import urllib.request
import json

# Storage for loaded data
loaded_data = {
    'obs_content': None,
    'obs_filename': None,
    'obs_path': None,
    'nav_content': None,
    'nav_filename': None,
    'nav_path': None,
}

# ============ NAVIGATION DOWNLOADER ============
class NavDownloader:
    """Multi-GNSS Navigation/Ephemeris Downloader"""
    
    @staticmethod
    def gps_week_from_date(year, doy):
        """Calculate GPS week and day-of-week from year and DOY"""
        from datetime import date, timedelta
        gps_epoch = date(1980, 1, 6)
        target = date(year, 1, 1) + timedelta(days=doy - 1)
        delta = (target - gps_epoch).days
        return delta // 7, delta % 7
    
    @staticmethod
    def parse_rinex_header(filepath):
        """Extract year, doy from RINEX file header"""
        from datetime import date
        try:
            with open(filepath, 'r', errors='ignore') as f:
                for line in f:
                    if 'TIME OF FIRST OBS' in line:
                        parts = line.split()
                        if len(parts) >= 6:
                            year = int(float(parts[0]))
                            month = int(float(parts[1]))
                            day = int(float(parts[2]))
                            doy = date(year, month, day).timetuple().tm_yday
                            return year, doy
                    if 'END OF HEADER' in line:
                        break
        except Exception as e:
            print(f"   Header parse error: {e}")
        return None, None
    
    @staticmethod
    def count_nav_satellites(content):
        """Count satellites per constellation in navigation file content"""
        if isinstance(content, bytes):
            content = content.decode('utf-8', errors='ignore')
        
        sats = {'G': set(), 'R': set(), 'E': set(), 'C': set(), 'J': set(), 'I': set()}
        
        for line in content.split('\n'):
            if len(line) >= 3:
                first_char = line[0]
                if first_char in sats:
                    try:
                        prn = line[1:3].strip()
                        if prn.isdigit():
                            sats[first_char].add(int(prn))
                    except:
                        pass
        
        return {k: len(v) for k, v in sats.items()}
    
    @staticmethod
    def format_sat_summary(counts):
        """Format satellite count summary"""
        names = {'G': 'GPS', 'R': 'GLO', 'E': 'GAL', 'C': 'BDS', 'J': 'QZS', 'I': 'NAV'}
        parts = []
        for sys, count in counts.items():
            if count > 0:
                parts.append(f"{names.get(sys, sys)}:{count}")
        return ", ".join(parts)
    
    @staticmethod
    def download(year, doy, output_dir, log_func=print, prefer_sp3=False):
        """Download ephemeris - tries multiple sources"""
        if prefer_sp3:
            result = NavDownloader.download_sp3(year, doy, output_dir, log_func)
            if result:
                return result
            log_func("   SP3 not available, trying BRDC...")
        
        result = NavDownloader.download_brdc_best(year, doy, output_dir, log_func)
        if result:
            return result
        
        if not prefer_sp3:
            log_func("   BRDC not available, trying SP3...")
            return NavDownloader.download_sp3(year, doy, output_dir, log_func)
        
        return None
    
    @staticmethod
    def download_brdc_best(year, doy, output_dir, log_func=print):
        """Download BRDC - tries multiple sources and picks the most complete"""
        import ssl
        
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        
        brdc_sources = [
            {"name": "BKG IGS", "url": f"https://igs.bkg.bund.de/root_ftp/IGS/BRDC/{year}/{doy:03d}/BRDC00IGS_R_{year}{doy:03d}0000_01D_MN.rnx.gz",
             "filename": f"BRDC00IGS_R_{year}{doy:03d}0000_01D_MN.rnx"},
            {"name": "DLR MGEX", "url": f"https://igs.bkg.bund.de/root_ftp/MGEX/BRDC/{year}/{doy:03d}/BRDM00DLR_S_{year}{doy:03d}0000_01D_MN.rnx.gz",
             "filename": f"BRDM00DLR_S_{year}{doy:03d}0000_01D_MN.rnx"},
            {"name": "IGN France", "url": f"https://igs.ign.fr/pub/igs/data/{year}/{doy:03d}/BRDC00IGS_R_{year}{doy:03d}0000_01D_MN.rnx.gz",
             "filename": f"BRDC00IGS_R_{year}{doy:03d}0000_01D_MN_ign.rnx"},
        ]
        
        out_path = Path(output_dir)
        candidates = []
        
        log_func("   Checking BRDC sources...")
        
        for source in brdc_sources:
            log_func(f"   ⏳ {source['name']}...")
            
            try:
                req = urllib.request.Request(source["url"])
                req.add_header('User-Agent', 'Mozilla/5.0 GNSS-Analysis')
                
                with urllib.request.urlopen(req, timeout=45, context=ctx) as resp:
                    data = resp.read()
                
                if len(data) < 1000:
                    continue
                
                try:
                    decompressed = gzip.decompress(data)
                except:
                    decompressed = data
                
                counts = NavDownloader.count_nav_satellites(decompressed)
                total = sum(counts.values())
                
                if total == 0:
                    continue
                
                summary = NavDownloader.format_sat_summary(counts)
                log_func(f"      ✓ {total} sats: {summary}")
                
                candidates.append({
                    'source': source,
                    'content': decompressed,
                    'counts': counts,
                    'total': total,
                    'summary': summary
                })
                
            except urllib.error.HTTPError as e:
                log_func(f"      ✗ HTTP {e.code}")
            except Exception as e:
                log_func(f"      ✗ {str(e)[:30]}")
        
        if not candidates:
            log_func("   ❌ No BRDC sources available")
            return None
        
        def score(c):
            num_systems = sum(1 for v in c['counts'].values() if v > 0)
            return (num_systems, c['total'])
        
        best = max(candidates, key=score)
        
        out_file = out_path / best['source']['filename']
        with open(out_file, 'wb') as f:
            f.write(best['content'])
        
        log_func(f"   ✅ Selected: {best['source']['name']}")
        log_func(f"   📊 Ephemeris: {best['summary']}")
        
        return out_file
    
    @staticmethod
    def download_sp3(year, doy, output_dir, log_func=print):
        """Download Multi-GNSS SP3 precise ephemeris"""
        import ssl
        
        week, dow = NavDownloader.gps_week_from_date(year, doy)
        
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        
        sp3_sources = [
            {"name": "ESA Final", "url": f"http://navigation-office.esa.int/products/gnss-products/{week}/ESA0MGNFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz"},
            {"name": "GFZ Final", "url": f"https://igs.bkg.bund.de/root_ftp/IGS/products/mgex/{week}/GFZ0MGXFIN_{year}{doy:03d}0000_01D_05M_ORB.SP3.gz"},
        ]
        
        out_path = Path(output_dir)
        log_func(f"   🔍 Looking for SP3 (Week {week})...")
        
        for source in sp3_sources:
            try:
                req = urllib.request.Request(source["url"])
                req.add_header('User-Agent', 'Mozilla/5.0 GNSS-Analysis')
                
                with urllib.request.urlopen(req, timeout=60, context=ctx) as resp:
                    data = resp.read()
                
                if len(data) < 10000:
                    continue
                
                try:
                    decompressed = gzip.decompress(data)
                except:
                    decompressed = data
                
                out_file = out_path / f"{source['name'].replace(' ', '_')}_{year}{doy:03d}.sp3"
                with open(out_file, 'wb') as f:
                    f.write(decompressed)
                
                log_func(f"   ✅ {source['name']}")
                return out_file
                
            except:
                pass
        
        log_func("   ❌ SP3 not available")
        return None


# ============ FILE INPUT WIDGETS ============
header = widgets.HTML("<h3>📡 CN0 Analysis - GNSS Signal Quality with Presets</h3>")

# === OBSERVATION FILE ===
obs_section = widgets.HTML("<b>Observation File</b> (required)")

obs_upload = widgets.FileUpload(
    accept='.obs,.rnx,.crx,.24o,.23o,.22o,.21o,.20o,.25o,.gz,.Z,*',
    multiple=False,
    description='Upload OBS',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

obs_path_input = widgets.Text(
    value='',
    placeholder='/path/to/observation.rnx',
    description='OBS Path:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='450px')
)

# === NAVIGATION FILE ===
nav_section = widgets.HTML("<b>Navigation/Ephemeris</b> (for elevation & skyplots)")

nav_upload = widgets.FileUpload(
    accept='.nav,.rnx,.24n,.24g,.25n,.sp3,.SP3,.gz,.Z,*',
    multiple=False,
    description='Upload NAV',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

nav_path_input = widgets.Text(
    value='',
    placeholder='/path/to/navigation.rnx or .sp3',
    description='NAV Path:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='450px')
)

auto_download_nav = widgets.Checkbox(
    value=True,
    description='Auto-download BRDC if missing',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='250px')
)

load_btn = widgets.Button(
    description='📥 Load Files',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

# === ANALYSIS CONFIG ===
config_section = widgets.HTML("<b>Analysis Configuration</b>")

# Preset dropdown with research-based configurations
preset_dropdown = widgets.Dropdown(
    options=[
        ('🔬 Full Analysis', 'full'),
        ('⚡ Quick Summary', 'quick'),
        ('📊 Interference Focus', 'interference'),
        ('🎯 Jamming Detection', 'jamming'),
        ('🛡️ Spoofing Check', 'spoofing'),
    ],
    value='full',
    description='Preset:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Preset help text with research-based thresholds
preset_help = widgets.HTML(value="""
<div style="font-size:11px; color:#666; margin-top:5px; padding:10px; background:#f8f9fa; border-radius:4px; border-left:3px solid #3182ce;">
<b>Preset Configurations (research-based thresholds):</b><br><br>
• <b>Full Analysis</b>: Complete analysis with all plots (sensitivity: 0.3, threshold: 8 dB)<br>
• <b>Quick Summary</b>: Fast overview, skips heavy plots (sensitivity: 0.5, threshold: 10 dB)<br>
• <b>Interference Focus</b>: Detect subtle interference &gt;4 dB (<i>ITU I/N=-6dB criterion</i>)<br>
• <b>Jamming Detection</b>: Rapid drops &gt;6 dB in &lt;3s (<i>Stanford GPS Lab</i>)<br>
• <b>Spoofing Check</b>: CN0 uniformity &amp; elevation anomalies (<i>GPS Solutions</i>)<br><br>
<i>Thresholds from: ITU-R M.1902-1, Stanford GPS Lab, GPS Solutions journal</i>
</div>
""")

elevation_slider = widgets.FloatSlider(
    value=5.0, min=0.0, max=30.0, step=1.0,
    description='Elevation Cutoff (°):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

time_bin_slider = widgets.IntSlider(
    value=60, min=10, max=300, step=10,
    description='Time Bin (sec):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

system_checks = {
    'G': widgets.Checkbox(value=True, description='GPS', layout=widgets.Layout(width='100px')),
    'R': widgets.Checkbox(value=True, description='GLONASS', layout=widgets.Layout(width='100px')),
    'E': widgets.Checkbox(value=True, description='Galileo', layout=widgets.Layout(width='100px')),
    'C': widgets.Checkbox(value=True, description='BeiDou', layout=widgets.Layout(width='100px')),
}

# === BUTTONS ===
analyze_btn = widgets.Button(
    description='🔬 Run CN0 Analysis',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

export_btn = widgets.Button(
    description='📥 Export Results',
    button_style='success',
    disabled=True,
    layout=widgets.Layout(width='150px')
)

clear_btn = widgets.Button(
    description='🗑️ Clear',
    button_style='danger',
    layout=widgets.Layout(width='100px')
)

# Progress & Status
progress = widgets.FloatProgress(
    value=0, min=0, max=1.0,
    description='Progress:',
    layout=widgets.Layout(width='400px', visibility='hidden')
)

status = widgets.HTML(value="<b>Status:</b> Ready - load files to begin")

# Output areas
info_out = widgets.Output()
results_out = widgets.Output()

# Store analysis results
analysis_results = {'data': None, 'figures': {}, 'report_html': '', 'lock_integrity': {}}


# ============ PRESET CONFIGURATIONS ============
# Research-based thresholds from:
# - ITU-R M.1902-1: I/N = -6 dB (1 dB noise floor increase) 
# - Stanford GPS Lab: CN0 min 27 dB-Hz, >6 dB drop in <3s = jamming
# - GPS Solutions: CN0 uniformity <2 dB std = spoofing indicator
# - MDPI Sensors: Multi-parameter detection

PRESET_CONFIGS = {
    'full': {
        'sensitivity': 0.3,
        'threshold': 8.0,
        'description': 'Complete analysis with all plots',
        'skip_heavy_plots': False,
    },
    'quick': {
        'sensitivity': 0.5,
        'threshold': 10.0,
        'description': 'Fast overview - skips heatmaps and per-satellite plots',
        'skip_heavy_plots': True,
    },
    'interference': {
        'sensitivity': 0.15,
        'threshold': 4.0,
        'description': 'Detect subtle interference >4 dB (ITU criterion)',
        'skip_heavy_plots': False,
    },
    'jamming': {
        'sensitivity': 0.2,
        'threshold': 6.0,
        'description': 'Optimized for jamming: rapid CN0 drops >6 dB',
        'skip_heavy_plots': False,
    },
    'spoofing': {
        'sensitivity': 0.1,
        'threshold': 5.0,
        'description': 'Focus on CN0 uniformity and elevation anomalies',
        'skip_heavy_plots': False,
    },
}


# ============ HTML REPORT GENERATOR ============
def generate_html_report(result, const_data, anomalies, lock_integrity_data=None):
    """Generate a comprehensive HTML report"""
    qs = result.quality_score
    
    # Unpack lock integrity data
    if lock_integrity_data:
        lock_score = lock_integrity_data.get('score', 0)
        total_cycle_slips = lock_integrity_data.get('total_cycle_slips', 0)
        total_data_gaps = lock_integrity_data.get('total_data_gaps', 0)
        slips_per_hour = lock_integrity_data.get('slips_per_hour', 0)
    else:
        lock_score = total_cycle_slips = total_data_gaps = slips_per_hour = 0
    
    html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>CN0 Analysis Report - {result.filename}</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; background: #f5f5f5; }}
        .container {{ max-width: 1200px; margin: 0 auto; background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }}
        h1 {{ color: #1a365d; border-bottom: 3px solid #3182ce; padding-bottom: 10px; }}
        h2 {{ color: #2c5282; margin-top: 30px; }}
        .score-box {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; text-align: center; margin: 20px 0; }}
        .score-value {{ font-size: 48px; font-weight: bold; }}
        .score-rating {{ font-size: 24px; opacity: 0.9; }}
        .metric-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(180px, 1fr)); gap: 15px; margin: 20px 0; }}
        .metric-card {{ background: #f7fafc; padding: 15px; border-radius: 8px; border-left: 4px solid #3182ce; }}
        .metric-value {{ font-size: 24px; font-weight: bold; color: #2d3748; }}
        .metric-label {{ color: #718096; font-size: 14px; }}
        .status-ok {{ color: #38a169; }}
        .status-warning {{ color: #d69e2e; }}
        .status-danger {{ color: #e53e3e; }}
        .anomaly {{ background: #fff5f5; border-left: 4px solid #e53e3e; padding: 10px; margin: 5px 0; border-radius: 4px; }}
        .footer {{ margin-top: 40px; padding-top: 20px; border-top: 1px solid #e2e8f0; color: #718096; font-size: 12px; }}
    </style>
</head>
<body>
<div class="container">
    <h1>📡 CN0 Analysis Report</h1>
    
    <h2>📁 File Information</h2>
    <div class="metric-grid">
        <div class="metric-card"><div class="metric-label">Filename</div><div class="metric-value" style="font-size:14px;">{result.filename}</div></div>
        <div class="metric-card"><div class="metric-label">RINEX Version</div><div class="metric-value">{result.rinex_version}</div></div>
        <div class="metric-card"><div class="metric-label">Duration</div><div class="metric-value">{result.duration_hours:.2f} h</div></div>
        <div class="metric-card"><div class="metric-label">Epochs</div><div class="metric-value">{result.epoch_count:,}</div></div>
    </div>
    
    <h2>🏆 Quality Score</h2>
    <div class="score-box">
        <div class="score-value">{qs.overall:.0f}/100</div>
        <div class="score-rating">{qs.rating}</div>
    </div>
    <div class="metric-grid">
        <div class="metric-card"><div class="metric-label">CN0 Quality</div><div class="metric-value">{qs.cn0_quality:.0f}</div></div>
        <div class="metric-card"><div class="metric-label">Availability</div><div class="metric-value">{qs.availability:.0f}</div></div>
        <div class="metric-card"><div class="metric-label">Continuity</div><div class="metric-value">{qs.continuity:.0f}</div></div>
        <div class="metric-card"><div class="metric-label">Stability</div><div class="metric-value">{qs.stability:.0f}</div></div>
        <div class="metric-card"><div class="metric-label">Diversity</div><div class="metric-value">{qs.diversity:.0f}</div></div>
        <div class="metric-card"><div class="metric-label">Lock Integrity</div><div class="metric-value">{lock_score:.0f}</div></div>
    </div>
    
    <h2>📶 Signal Quality</h2>
    <div class="metric-grid">
        <div class="metric-card"><div class="metric-label">Average CN0</div><div class="metric-value">{result.avg_cn0:.1f} dB-Hz</div></div>
        <div class="metric-card"><div class="metric-label">Std Deviation</div><div class="metric-value">{result.cn0_std_dev:.1f} dB-Hz</div></div>
        <div class="metric-card"><div class="metric-label">Range</div><div class="metric-value">{result.min_cn0:.1f} - {result.max_cn0:.1f}</div></div>
    </div>
    
    <h2>🔓 Lock Integrity</h2>
    <div class="metric-grid">
        <div class="metric-card"><div class="metric-label">Cycle Slips</div><div class="metric-value">{total_cycle_slips}</div><div style="font-size:11px;">{slips_per_hour:.1f}/hour</div></div>
        <div class="metric-card"><div class="metric-label">Data Gaps</div><div class="metric-value">{total_data_gaps}</div></div>
        <div class="metric-card"><div class="metric-label">Lock Score</div><div class="metric-value {'status-ok' if lock_score >= 70 else 'status-warning' if lock_score >= 50 else 'status-danger'}">{lock_score:.0f}/100</div></div>
    </div>
    
    <h2>🛡️ Threat Assessment</h2>
    <div class="metric-grid">
        <div class="metric-card"><div class="metric-label">Jamming</div><div class="metric-value {'status-danger' if result.jamming_detected else 'status-ok'}">{'🚨 DETECTED' if result.jamming_detected else '✅ None'}</div></div>
        <div class="metric-card"><div class="metric-label">Spoofing</div><div class="metric-value {'status-danger' if result.spoofing_detected else 'status-ok'}">{'🚨 DETECTED' if result.spoofing_detected else '✅ None'}</div></div>
        <div class="metric-card"><div class="metric-label">Interference</div><div class="metric-value {'status-warning' if result.interference_detected else 'status-ok'}">{'⚠️ Yes' if result.interference_detected else '✅ None'}</div></div>
    </div>
    
    <h2>⚠️ Anomalies ({len(anomalies) if anomalies else 0})</h2>
    {'<p>No anomalies detected.</p>' if not anomalies else ''.join([f"<div class='anomaly'><b>{a.get('anomaly_type', 'Unknown')}</b> ({a.get('severity', 'low')}) - {a.get('start_time', '')}</div>" for a in anomalies[:20]])}
    
    <h2>📝 Summary</h2>
    <p>{result.summary}</p>
    
    <div class="footer">
        <p>Generated by CN0 Analysis Widget | {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
    </div>
</div>
</body>
</html>
    """
    
    return html


# ============ HANDLERS ============
def load_files(btn):
    """Load files from paths OR upload widgets"""
    with info_out:
        clear_output()
        print("📥 Loading files...")
        
        obs_loaded = False
        nav_loaded = False
        
        # ===== OBSERVATION FILE =====
        if obs_path_input.value.strip():
            path = obs_path_input.value.strip()
            print(f"\n📂 Loading OBS from path: {path}")
            
            if os.path.exists(path):
                try:
                    with open(path, 'rb') as f:
                        content = f.read()
                    
                    name = os.path.basename(path)
                    if name.lower().endswith('.gz'):
                        content = gzip.decompress(content)
                    
                    loaded_data['obs_content'] = content
                    loaded_data['obs_filename'] = name
                    loaded_data['obs_path'] = path
                    print(f"✅ Loaded: {name} ({len(content)/1024/1024:.1f} MB)")
                    obs_loaded = True
                except Exception as e:
                    print(f"❌ Error: {e}")
            else:
                print(f"❌ File not found: {path}")
        
        if not obs_loaded and obs_upload.value:
            print(f"\n📤 Loading OBS from upload widget...")
            try:
                if len(obs_upload.value) > 0:
                    if isinstance(obs_upload.value, dict):
                        name = list(obs_upload.value.keys())[0]
                        content = obs_upload.value[name]['content']
                    else:
                        fi = obs_upload.value[0]
                        name = getattr(fi, 'name', fi.get('name', 'unknown'))
                        content = getattr(fi, 'content', fi.get('content', b''))
                    
                    if content:
                        if name.lower().endswith('.gz'):
                            content = gzip.decompress(content)
                        
                        loaded_data['obs_content'] = content
                        loaded_data['obs_filename'] = name
                        print(f"✅ Loaded: {name} ({len(content)/1024/1024:.1f} MB)")
                        obs_loaded = True
            except Exception as e:
                print(f"❌ Upload error: {e}")
        
        if not obs_loaded:
            print("\n❌ No observation file loaded!")
            status.value = "<b>Status:</b> <span style='color:red'>Missing observation file</span>"
            return
        
        # ===== NAVIGATION FILE =====
        if nav_path_input.value.strip():
            path = nav_path_input.value.strip()
            print(f"\n📂 Loading NAV from path: {path}")
            
            if os.path.exists(path):
                try:
                    with open(path, 'rb') as f:
                        content = f.read()
                    
                    name = os.path.basename(path)
                    if name.lower().endswith('.gz'):
                        content = gzip.decompress(content)
                    
                    loaded_data['nav_content'] = content
                    loaded_data['nav_filename'] = name
                    loaded_data['nav_path'] = path
                    print(f"✅ Loaded: {name}")
                    nav_loaded = True
                except Exception as e:
                    print(f"❌ Error: {e}")
        
        if not nav_loaded and nav_upload.value:
            try:
                if len(nav_upload.value) > 0:
                    if isinstance(nav_upload.value, dict):
                        name = list(nav_upload.value.keys())[0]
                        content = nav_upload.value[name]['content']
                    else:
                        fi = nav_upload.value[0]
                        name = getattr(fi, 'name', fi.get('name', 'unknown'))
                        content = getattr(fi, 'content', fi.get('content', b''))
                    
                    if content:
                        if name.lower().endswith('.gz'):
                            content = gzip.decompress(content)
                        
                        loaded_data['nav_content'] = content
                        loaded_data['nav_filename'] = name
                        print(f"✅ Loaded: {name}")
                        nav_loaded = True
            except Exception as e:
                print(f"❌ Upload error: {e}")
        
        # Try auto-download
        if not nav_loaded and auto_download_nav.value and obs_loaded:
            print("\n🌐 Auto-downloading navigation file...")
            year, doy = None, None
            if loaded_data.get('obs_path'):
                year, doy = NavDownloader.parse_rinex_header(loaded_data['obs_path'])
            elif loaded_data.get('obs_content'):
                header = loaded_data['obs_content'][:8000].decode('utf-8', errors='ignore')
                for line in header.split('\n'):
                    if 'TIME OF FIRST OBS' in line:
                        from datetime import date
                        parts = line.split()
                        if len(parts) >= 6:
                            year = int(float(parts[0]))
                            month = int(float(parts[1]))
                            day = int(float(parts[2]))
                            doy = date(year, month, day).timetuple().tm_yday
                        break
            
            if year and doy:
                print(f"   Date: Year {year}, DOY {doy}")
                temp_dir = tempfile.gettempdir()
                nav_path = NavDownloader.download(year, doy, temp_dir, log_func=print)
                
                if nav_path and nav_path.exists():
                    with open(nav_path, 'rb') as f:
                        loaded_data['nav_content'] = f.read()
                    loaded_data['nav_filename'] = nav_path.name
                    loaded_data['nav_path'] = str(nav_path)
                    nav_loaded = True
        
        # Summary
        print("\n" + "="*50)
        print("📊 LOADED FILES:")
        print(f"   OBS: {loaded_data['obs_filename'] or 'None'}")
        print(f"   NAV: {loaded_data['nav_filename'] or 'None (elevations estimated)'}")
        
        if nav_loaded:
            status.value = "<b>Status:</b> <span style='color:green'>✓ Files loaded with ephemeris</span>"
        else:
            status.value = "<b>Status:</b> <span style='color:orange'>⚠ No NAV - elevations estimated</span>"
        
        analyze_btn.disabled = False


def run_analysis(btn):
    """Run CN0 analysis with preset configuration"""
    if not loaded_data['obs_content']:
        with info_out:
            print("❌ No observation file loaded!")
        return
    
    with results_out:
        clear_output()
    
    progress.layout.visibility = 'visible'
    progress.value = 0.05
    status.value = "<b>Status:</b> <span style='color:blue'>⏳ Starting analysis...</span>"
    analyze_btn.disabled = True
    
    with results_out:
        print("🔬 Starting CN0 Analysis...")
        
        # Get config
        elev_cutoff = elevation_slider.value
        time_bin = int(time_bin_slider.value)
        systems = [k for k, v in system_checks.items() if v.value]
        preset = preset_dropdown.value
        
        # Get preset configuration
        preset_cfg = PRESET_CONFIGS.get(preset, PRESET_CONFIGS['full'])
        
        print(f"\n📋 Preset: {preset.upper()}")
        print(f"   {preset_cfg['description']}")
        print(f"   Sensitivity: {preset_cfg['sensitivity']}")
        print(f"   Threshold: {preset_cfg['threshold']} dB")
        print(f"\n📋 Configuration:")
        print(f"   Elevation cutoff: {elev_cutoff}°")
        print(f"   Time bin: {time_bin}s")
        print(f"   Systems: {', '.join(systems)}")
        
        progress.value = 0.1
        status.value = "<b>Status:</b> <span style='color:blue'>⏳ Preparing files...</span>"
        
        # Write temp files
        temp_dir = tempfile.mkdtemp()
        obs_path = os.path.join(temp_dir, loaded_data['obs_filename'])
        
        with open(obs_path, 'wb') as f:
            f.write(loaded_data['obs_content'])
        
        nav_path = None
        if loaded_data['nav_content']:
            nav_path = os.path.join(temp_dir, loaded_data['nav_filename'])
            with open(nav_path, 'wb') as f:
                f.write(loaded_data['nav_content'])
        
        progress.value = 0.3
        
        # Try Rust library
        try:
            import geoveil_cn0 as gcn0
            print(f"\n🦀 Using geoveil_cn0 v{gcn0.VERSION}")
            
            status.value = "<b>Status:</b> <span style='color:blue'>⏳ Running Rust analysis...</span>"
            
            # Create config with preset-based parameters
            config = gcn0.AnalysisConfig(
                min_elevation=elev_cutoff,
                time_bin=int(time_bin),
                detect_anomalies=True,
                anomaly_sensitivity=preset_cfg['sensitivity'],
                interference_threshold_db=preset_cfg['threshold'],
                verbose=True,
                nav_file=nav_path if nav_path else None,
            )
            
            analyzer = gcn0.CN0Analyzer(config)
            
            if nav_path:
                print(f"   Using navigation: {os.path.basename(nav_path)}")
                result = analyzer.analyze_with_nav(obs_path, nav_path)
            else:
                result = analyzer.analyze_file(obs_path)
            
            progress.value = 0.8
            
            # Display results with preset mode
            display_rust_results(result, preset, preset_cfg)
            analysis_results['data'] = result
            export_btn.disabled = False
            
        except ImportError as e:
            print(f"\n⚠️ Rust library not available: {e}")
            print("   Install with: pip install ./geoveil-cn0")
            
        except Exception as e:
            print(f"\n❌ Analysis error: {e}")
            import traceback
            traceback.print_exc()
        
        progress.value = 1.0
        status.value = "<b>Status:</b> <span style='color:green'>✓ Analysis complete</span>"
        analyze_btn.disabled = False


def display_rust_results(result, preset='full', preset_cfg=None):
    """Display results with preset-specific output"""
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import numpy as np
    import pandas as pd  # For robust timestamp parsing
    
    if preset_cfg is None:
        preset_cfg = PRESET_CONFIGS.get(preset, PRESET_CONFIGS['full'])
    
    quick_mode = preset_cfg.get('skip_heavy_plots', False)
    
    figures = {}
    report_lines = []
    const_data = []
    
    # Get anomalies
    try:
        anomalies = result.get_anomalies()
    except:
        anomalies = []
    
    def add_report(text):
        report_lines.append(text)
        print(text)
    
    add_report("\n" + "="*70)
    add_report("📊 CN0 ANALYSIS RESULTS")
    add_report("="*70)
    
    # File info
    add_report(f"\n📁 File: {result.filename}")
    add_report(f"   RINEX Version: {result.rinex_version}")
    add_report(f"   Duration: {result.duration_hours:.2f} hours ({result.epoch_count} epochs)")
    if result.station_name:
        add_report(f"   Station: {result.station_name}")
    add_report(f"   Constellations: {', '.join(result.constellations)}")
    
    # Quality Score
    qs = result.quality_score
    
    # === CALCULATE LOCK INTEGRITY ===
    total_cycle_slips = 0
    total_data_gaps = 0
    total_satellites = 0
    
    for const_name in result.constellations:
        cs = result.get_constellation_summary(const_name)
        if cs:
            total_cycle_slips += int(cs.get('cycle_slips', 0))
            total_data_gaps += int(cs.get('data_gaps', 0))
            total_satellites += int(cs.get('satellites_observed', cs.get('satellite_count', 0)))
    
    duration_hours = max(result.duration_hours, 0.01)
    slips_per_hour = total_cycle_slips / duration_hours
    gaps_per_hour = total_data_gaps / duration_hours
    
    if total_satellites > 0:
        slips_per_sat_hour = slips_per_hour / total_satellites
        gaps_per_sat_hour = gaps_per_hour / total_satellites
        slip_score = max(0, min(100, 100 - (slips_per_sat_hour * 50)))
        gap_score = max(0, min(100, 100 - (gaps_per_sat_hour * 25)))
        lock_integrity_score = (slip_score * 0.6 + gap_score * 0.4)
    else:
        lock_integrity_score = 0
        slips_per_sat_hour = 0
    
    add_report(f"\n🏆 QUALITY SCORE: {qs.overall:.0f}/100 ({qs.rating})")
    add_report(f"   CN0 Quality:   {qs.cn0_quality:.0f}")
    add_report(f"   Availability:  {qs.availability:.0f}")
    add_report(f"   Continuity:    {qs.continuity:.0f}")
    add_report(f"   Stability:     {qs.stability:.0f}")
    add_report(f"   Diversity:     {qs.diversity:.0f}")
    add_report(f"   Lock Integrity: {lock_integrity_score:.0f}")
    
    # Signal quality
    add_report(f"\n📶 SIGNAL QUALITY:")
    add_report(f"   Average CN0: {result.avg_cn0:.1f} dB-Hz")
    add_report(f"   Std Dev: {result.cn0_std_dev:.1f} dB-Hz")
    add_report(f"   Range: {result.min_cn0:.1f} - {result.max_cn0:.1f} dB-Hz")
    
    # Lock Loss
    add_report(f"\n🔓 LOCK INTEGRITY:")
    add_report(f"   Cycle Slips: {total_cycle_slips} ({slips_per_hour:.1f}/hour)")
    add_report(f"   Data Gaps: {total_data_gaps} ({gaps_per_hour:.1f}/hour)")
    add_report(f"   Score: {lock_integrity_score:.0f}/100")
    
    # === PLOT 1: Quality Score Radar (with Lock Integrity) ===
    try:
        categories = ['CN0 Quality', 'Availability', 'Continuity', 'Stability', 'Diversity', 'Lock Integrity']
        values = [qs.cn0_quality, qs.availability, qs.continuity, qs.stability, qs.diversity, lock_integrity_score]
        values.append(values[0])  # Close polygon
        categories.append(categories[0])
        
        fig_radar = go.Figure()
        fig_radar.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            fill='toself',
            fillcolor='rgba(99, 110, 250, 0.3)',
            line=dict(color='rgb(99, 110, 250)', width=2),
            name=f'Score: {qs.overall:.0f}'
        ))
        fig_radar.update_layout(
            polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
            title=f"Quality Score: {qs.overall:.0f}/100 ({qs.rating})",
            showlegend=True,
            height=450
        )
        figures['quality_radar'] = fig_radar
        print("\n📊 Quality Score Radar:")
        fig_radar.show()
    except Exception as e:
        print(f"   (Radar chart error: {e})")
    
    # Constellation summaries
    add_report("\n🛰️ CONSTELLATION SUMMARY:")
    sys_names = {'GPS': 'GPS', 'GLONASS': 'GLONASS', 'Galileo': 'Galileo', 
                 'BeiDou': 'BeiDou', 'QZSS': 'QZSS', 'IRNSS': 'NavIC'}
    sys_colors = {'GPS': '#1f77b4', 'GLONASS': '#ff7f0e', 'Galileo': '#2ca02c', 
                  'BeiDou': '#d62728', 'QZSS': '#9467bd', 'IRNSS': '#8c564b'}
    
    for const_name in result.constellations:
        cs = result.get_constellation_summary(const_name)
        if cs:
            name = sys_names.get(const_name, const_name)
            slips_ph = int(cs['cycle_slips']) / max(result.duration_hours, 0.01)
            add_report(f"\n   {name} ({cs['constellation']}):")
            add_report(f"      Satellites: {cs['satellites_observed']}/{cs['satellites_expected']}")
            add_report(f"      CN0: {cs['cn0_mean']} ± {cs['cn0_std']} dB-Hz")
            add_report(f"      Cycle Slips: {cs['cycle_slips']} ({slips_ph:.2f}/hour)")
            const_data.append({
                'name': name, 'code': cs['constellation'],
                'sats': int(cs['satellites_observed']), 'expected': int(cs['satellites_expected']),
                'cn0_mean': float(cs['cn0_mean']), 'cn0_std': float(cs['cn0_std']),
                'slips': int(cs['cycle_slips']), 'gaps': int(cs['data_gaps']),
                'availability': float(cs['availability_ratio'])
            })
    
    # === PLOT 2: Constellation Bar Chart ===
    if const_data:
        try:
            fig_const = make_subplots(rows=1, cols=2, 
                                       subplot_titles=('Mean CN0 by Constellation', 'Satellite Count'))
            
            names = [d['name'] for d in const_data]
            colors = [sys_colors.get(d['name'], '#999') for d in const_data]
            
            fig_const.add_trace(go.Bar(
                x=names, y=[d['cn0_mean'] for d in const_data],
                error_y=dict(type='data', array=[d['cn0_std'] for d in const_data]),
                marker_color=colors, name='CN0',
                text=[f"{d['cn0_mean']:.1f}" for d in const_data],
                textposition='outside'
            ), row=1, col=1)
            
            fig_const.add_trace(go.Bar(
                x=names, y=[d['sats'] for d in const_data],
                marker_color=colors, name='Observed',
            ), row=1, col=2)
            
            fig_const.add_hline(y=35, line_dash="dash", line_color="orange", row=1, col=1)
            fig_const.add_hline(y=25, line_dash="dash", line_color="red", row=1, col=1)
            
            fig_const.update_layout(height=400, title="Constellation Overview", showlegend=False)
            figures['constellation_overview'] = fig_const
            print("\n📊 Constellation Overview:")
            fig_const.show()
        except Exception as e:
            print(f"   (Constellation chart error: {e})")
    
    # === PLOT 3: CN0 Timeseries ===
    try:
        ts_data = result.get_timeseries_data()
        if ts_data and len(ts_data.get('timestamps', [])) > 0:
            fig_ts = make_subplots(rows=2, cols=1, shared_xaxes=True,
                                    subplot_titles=('Mean CN0 Over Time', 'Satellite Count'),
                                    vertical_spacing=0.1)
            
            timestamps = ts_data['timestamps']
            cn0_values = ts_data['cn0_mean']
            sat_counts = ts_data.get('satellite_counts', [])
            
            fig_ts.add_trace(go.Scatter(
                x=timestamps, y=cn0_values,
                mode='lines', name='Mean CN0',
                line=dict(color='blue', width=1.5),
                fill='tozeroy', fillcolor='rgba(0,100,255,0.1)'
            ), row=1, col=1)
            
            fig_ts.add_hline(y=35, line_dash="dash", line_color="orange", row=1, col=1)
            fig_ts.add_hline(y=25, line_dash="dash", line_color="red", row=1, col=1)
            
            if sat_counts and len(sat_counts) == len(timestamps):
                fig_ts.add_trace(go.Scatter(
                    x=timestamps, y=sat_counts,
                    mode='lines', name='Satellites',
                    line=dict(color='green', width=1.5),
                ), row=2, col=1)
            
            fig_ts.update_layout(height=500, title='CN0 Timeseries', showlegend=True)
            figures['timeseries'] = fig_ts
            print("\n📊 CN0 Timeseries:")
            fig_ts.show()
    except Exception as e:
        print(f"   (Timeseries chart error: {e})")
    
    # === PLOT 4: CN0 Heatmap (skip in quick mode) ===
    if quick_mode:
        print("\n📊 CN0 Heatmap: Skipped (quick mode)")
    else:
        try:
            result_json = json.loads(result.to_json())
            sat_timeseries = result_json.get('timeseries', {}).get('satellite_timeseries', {})
            
            if sat_timeseries and len(sat_timeseries) > 0:
                all_times = set()
                for sat_id, sat_data in sat_timeseries.items():
                    if isinstance(sat_data, dict):
                        series = sat_data.get('cn0_series', sat_data.get('series', []))
                        if isinstance(series, list):
                            for point in series:
                                if isinstance(point, dict):
                                    all_times.add(point.get('timestamp', point.get('time', '')))
                
                all_times = sorted([t for t in all_times if t])
                
                if all_times:
                    max_time_points = 400
                    if len(all_times) > max_time_points:
                        step = len(all_times) // max_time_points
                        all_times = all_times[::step]
                    
                    def sat_sort_key(s):
                        if len(s) >= 2:
                            sys = s[0]
                            try:
                                prn = int(s[1:])
                            except:
                                prn = 0
                            sys_order = {'G': 0, 'R': 1, 'E': 2, 'C': 3}
                            return (sys_order.get(sys, 9), prn)
                        return (9, 0)
                    
                    all_satellites = sorted(sat_timeseries.keys(), key=sat_sort_key, reverse=True)
                    
                    z_matrix = []
                    sat_labels = []
                    
                    for sat in all_satellites:
                        sat_data = sat_timeseries.get(sat, {})
                        if isinstance(sat_data, dict):
                            cn0_series = sat_data.get('cn0_series', sat_data.get('series', []))
                        else:
                            continue
                        
                        cn0_by_time = {}
                        if isinstance(cn0_series, list):
                            for p in cn0_series:
                                if isinstance(p, dict):
                                    t = p.get('timestamp', p.get('time', ''))
                                    v = p.get('value', p.get('cn0', None))
                                    if t and v is not None:
                                        cn0_by_time[t] = v
                        
                        row = [cn0_by_time.get(t, None) for t in all_times]
                        valid_count = sum(1 for v in row if v is not None)
                        if valid_count > len(all_times) * 0.05:
                            z_matrix.append(row)
                            sat_labels.append(sat)
                    
                    if z_matrix:
                        time_labels = pd.to_datetime(all_times)
                        
                        fig_heat = go.Figure(data=go.Heatmap(
                            z=z_matrix,
                            x=time_labels,
                            y=sat_labels,
                            colorscale='Viridis',
                            zmin=25, zmax=55,
                            colorbar=dict(title='C/N₀<br>(dB-Hz)'),
                            hoverongaps=False,
                        ))
                        
                        fig_heat.update_layout(
                            title=f"C/N₀ Heatmap - Time vs Satellite",
                            xaxis_title="Time",
                            yaxis_title="Satellite PRN",
                            height=max(400, len(sat_labels) * 18),
                            width=1100,
                        )
                        
                        figures['cn0_heatmap'] = fig_heat
                        print(f"\n📊 C/N₀ Heatmap ({len(sat_labels)} satellites):")
                        fig_heat.show()
        except Exception as e:
            print(f"   (Heatmap error: {e})")
    
    # === PLOT 5: Per-Constellation Timeseries (skip in quick mode) ===
    if quick_mode:
        print("\n📊 Per-Constellation Timeseries: Skipped (quick mode)")
    else:
        try:
            result_json = json.loads(result.to_json())
            sat_timeseries = result_json.get('timeseries', {}).get('satellite_timeseries', {})
            
            if sat_timeseries:
                by_constellation = {}
                sys_code_map = {'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 'C': 'BeiDou'}
                
                for sat_id, sat_data in sat_timeseries.items():
                    const = sat_id[0]
                    const_name = sys_code_map.get(const, const)
                    if const_name not in by_constellation:
                        by_constellation[const_name] = {}
                    by_constellation[const_name][sat_id] = sat_data
                
                sat_colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A',
                             '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
                
                for const_name, satellites in by_constellation.items():
                    if not satellites:
                        continue
                    
                    fig_const_ts = go.Figure()
                    
                    for i, (sat_id, sat_data) in enumerate(sorted(satellites.items())):
                        cn0_series = sat_data.get('cn0_series', sat_data.get('series', []))
                        if not cn0_series:
                            continue
                        
                        times = [p.get('timestamp', p.get('time', '')) for p in cn0_series if isinstance(p, dict)]
                        values = [p.get('value', p.get('cn0', 0)) for p in cn0_series if isinstance(p, dict)]
                        
                        if times and values:
                            fig_const_ts.add_trace(go.Scatter(
                                x=pd.to_datetime(times),
                                y=values,
                                mode='lines',
                                name=sat_id,
                                line=dict(width=1.2, color=sat_colors[i % len(sat_colors)]),
                            ))
                    
                    fig_const_ts.add_hline(y=35, line_dash="dash", line_color="orange")
                    fig_const_ts.add_hline(y=25, line_dash="dash", line_color="red")
                    
                    fig_const_ts.update_layout(
                        title=f"📡 {const_name} - C/N₀ by Satellite",
                        xaxis_title="Time (UTC)",
                        yaxis_title="C/N₀ (dB-Hz)",
                        height=450,
                        yaxis=dict(range=[20, 60]),
                        legend=dict(orientation='h', y=1.02, font=dict(size=9)),
                    )
                    
                    figures[f'timeseries_{const_name.lower()}'] = fig_const_ts
                    print(f"\n📊 {const_name} Timeseries ({len(satellites)} satellites):")
                    fig_const_ts.show()
        except Exception as e:
            print(f"   (Per-constellation timeseries error: {e})")
    
    # === PLOT 6: Skyplot ===
    try:
        skyplot_data = result.get_skyplot_data()
        if skyplot_data:
            traces = skyplot_data if isinstance(skyplot_data, list) else skyplot_data.get('traces', [])
            
            if traces:
                fig_sky = go.Figure()
                
                for trace in traces:
                    if not isinstance(trace, dict):
                        continue
                    
                    sat_id = trace.get('name', trace.get('satellite', ''))
                    
                    # Parse CSV strings
                    def parse_csv(value):
                        if isinstance(value, list):
                            return [float(v) for v in value if v is not None]
                        if isinstance(value, str):
                            return [float(v.strip()) for v in value.split(',') if v.strip()]
                        return []
                    
                    elevations = parse_csv(trace.get('elevations', trace.get('elevation', [])))
                    azimuths = parse_csv(trace.get('azimuths', trace.get('azimuth', [])))
                    cn0_vals = parse_csv(trace.get('cn0_values', trace.get('cn0', [])))
                    
                    if elevations and azimuths:
                        r_vals = [90.0 - el for el in elevations]
                        min_len = min(len(r_vals), len(azimuths))
                        cn0_list = cn0_vals[:min_len] if cn0_vals else [40.0] * min_len
                        
                        fig_sky.add_trace(go.Scatterpolar(
                            r=r_vals[:min_len],
                            theta=azimuths[:min_len],
                            mode='markers+lines',
                            marker=dict(
                                size=6,
                                color=cn0_list,
                                colorscale='RdYlGn',
                                cmin=25, cmax=55,
                            ),
                            line=dict(width=1.5),
                            name=str(sat_id),
                        ))
                
                fig_sky.update_layout(
                    polar=dict(
                        radialaxis=dict(visible=True, range=[90, 0],
                                       tickvals=[0, 30, 60, 90],
                                       ticktext=['90°', '60°', '30°', '0°']),
                        angularaxis=dict(direction='clockwise', rotation=90,
                                        tickvals=[0, 90, 180, 270],
                                        ticktext=['N', 'E', 'S', 'W'])
                    ),
                    title="🛰️ Skyplot (Color = CN0)",
                    showlegend=False,
                    height=550,
                    width=600
                )
                
                figures['skyplot'] = fig_sky
                print(f"\n📊 Skyplot ({len(traces)} satellites):")
                fig_sky.show()
    except Exception as e:
        print(f"   (Skyplot error: {e})")
    
    # === PLOT 7: Anomaly Timeline (FIXED with pd.to_datetime) ===
    if anomalies:
        add_report(f"\n⚠️ ANOMALIES DETECTED: {len(anomalies)}")
        
        try:
            fig_anom = go.Figure()
            
            # Case-insensitive severity colors
            severity_colors = {
                'critical': '#ef4444', 'high': '#f97316',
                'medium': '#eab308', 'low': '#22c55e'
            }
            
            # Parse anomalies using pd.to_datetime (robust!)
            valid_anomalies = []
            for a in anomalies[:500]:
                try:
                    ts_str = a.get('start_time') or a.get('timestamp') or ''
                    if ts_str:
                        # Use pandas for robust parsing
                        ts = pd.to_datetime(ts_str, errors='coerce')
                        if pd.notna(ts):
                            valid_anomalies.append({
                                'time': ts,
                                'severity': str(a.get('severity', 'low')).lower(),
                                'cn0_drop': float(a.get('cn0_drop', a.get('cn0_drop_db', 0)) or 0),
                                'type': a.get('anomaly_type', a.get('type', 'Unknown')),
                                'description': a.get('description', ''),
                                'confidence': float(a.get('confidence', 0.5) or 0.5),
                            })
                except:
                    continue
            
            if valid_anomalies:
                for severity in ['critical', 'high', 'medium', 'low']:
                    sev_data = [d for d in valid_anomalies if d['severity'] == severity]
                    if sev_data:
                        fig_anom.add_trace(go.Scatter(
                            x=[d['time'] for d in sev_data],
                            y=[d['cn0_drop'] for d in sev_data],
                            mode='markers',
                            marker=dict(
                                size=[8 + d['confidence'] * 10 for d in sev_data],
                                color=severity_colors.get(severity, '#888'),
                                opacity=0.7,
                            ),
                            name=f"{severity.capitalize()} ({len(sev_data)})",
                            text=[f"{d['type']}<br>{d['description']}" for d in sev_data],
                            hovertemplate="Time: %{x}<br>CN0 Drop: %{y:.1f} dB<br>%{text}<extra></extra>"
                        ))
                
                # Add threshold lines based on preset
                if preset == 'jamming':
                    fig_anom.add_hline(y=6.0, line_dash="dash", line_color="red",
                                      annotation_text="Jamming (6 dB)")
                elif preset == 'interference':
                    fig_anom.add_hline(y=4.0, line_dash="dash", line_color="orange",
                                      annotation_text="Interference (4 dB)")
                
                fig_anom.update_layout(
                    title=f"⚠️ Anomaly Timeline ({len(valid_anomalies)} events)",
                    xaxis_title="Time (UTC)",
                    xaxis=dict(type='date'),
                    yaxis_title="CN0 Drop (dB)",
                    height=450,
                    legend=dict(orientation='h', y=1.1)
                )
                figures['anomalies'] = fig_anom
                print(f"\n📊 Anomaly Timeline ({len(valid_anomalies)} events):")
                fig_anom.show()
                
                # === PRESET-SPECIFIC ANALYSIS ===
                if preset == 'jamming':
                    jamming_events = [a for a in valid_anomalies if a['cn0_drop'] >= 6.0]
                    print(f"\n🎯 JAMMING ANALYSIS:")
                    print(f"   Events with >6 dB drop: {len(jamming_events)}")
                    if jamming_events:
                        max_drop = max(a['cn0_drop'] for a in jamming_events)
                        print(f"   Maximum CN0 drop: {max_drop:.1f} dB")
                        if max_drop >= 10:
                            print(f"   ⚠️ SEVERE JAMMING detected (>10 dB drop)")
                
                elif preset == 'spoofing':
                    print(f"\n🛡️ SPOOFING ANALYSIS:")
                    if result.cn0_std_dev < 2.0:
                        print(f"   ⚠️ CN0 variance LOW ({result.cn0_std_dev:.2f} dB) - possible spoofing")
                    else:
                        print(f"   ✅ CN0 variance normal ({result.cn0_std_dev:.2f} dB)")
                    if result.avg_cn0 > 50:
                        print(f"   ⚠️ Average CN0 elevated ({result.avg_cn0:.1f} dB-Hz) - possible high-power spoofing")
                
                elif preset == 'interference':
                    print(f"\n📊 INTERFERENCE ANALYSIS:")
                    subtle = [a for a in valid_anomalies if 4.0 <= a['cn0_drop'] < 6.0]
                    moderate = [a for a in valid_anomalies if a['cn0_drop'] >= 6.0]
                    print(f"   Subtle interference (4-6 dB): {len(subtle)}")
                    print(f"   Moderate/severe (≥6 dB): {len(moderate)}")
            else:
                print(f"\n⚠️ {len(anomalies)} anomalies but could not parse timestamps")
        except Exception as e:
            print(f"   (Anomaly chart error: {e})")
            import traceback
            traceback.print_exc()
    else:
        add_report("\n✅ No anomalies detected")
    
    print(f"\n🛡️ THREAT ASSESSMENT:")
    
    # === IMPROVED THREAT DETECTION ===
    # Don't blindly trust the library's spoofing flag - it has false positives
    # When ephemeris doesn't cover all satellites (especially BeiDou), it triggers falsely
    
    # Jamming: Only if low CN0 + library flag
    jamming_detected = result.jamming_detected and result.mean_cn0 < 35.0
    
    # Spoofing: Check per-constellation std to avoid false positives from incomplete ephemeris
    constellation_stds = []
    for sys_name in ['GPS', 'GLONASS', 'Galileo', 'BeiDou']:
        stats = result.get_constellation_summary(sys_name)
        if stats:
            try:
                std_cn0 = float(stats.get('std_cn0', stats.get('cn0_std', 5.0)))
                constellation_stds.append(std_cn0)
            except:
                pass
    
    avg_constellation_std = sum(constellation_stds) / len(constellation_stds) if constellation_stds else 5.0
    
    # Real spoofing indicators:
    # 1. Very low CN0 std across constellations (< 2 dB) - signals too uniform
    # 2. AND elevated average CN0 (> 50 dB-Hz) - spoofer typically overpowers
    # 3. AND overall std also low
    spoofing_indicators = []
    if avg_constellation_std < 2.0:
        spoofing_indicators.append("CN0 uniformity suspiciously low")
    if result.mean_cn0 > 50.0:
        spoofing_indicators.append("CN0 elevated (possible high-power signal)")
    if result.cn0_std_dev < 1.0:
        spoofing_indicators.append("Overall signal variance very low")
    
    # Only flag spoofing if multiple indicators present
    spoofing_suspicious = len(spoofing_indicators) >= 2
    
    # If library says spoofing but our checks don't agree, it's likely false positive
    if result.spoofing_detected and not spoofing_suspicious:
        print(f"   Jamming:      {'🚨 DETECTED' if jamming_detected else '✅ None'}")
        print(f"   Spoofing:     ⚠️ Flag raised (likely false positive - incomplete ephemeris)")
        print(f"   Interference: {'⚠️ Detected' if result.interference_detected else '✅ None'}")
    else:
        print(f"   Jamming:      {'🚨 DETECTED' if jamming_detected else '✅ None'}")
        print(f"   Spoofing:     {'🚨 DETECTED' if spoofing_suspicious else '✅ None'}")
        print(f"   Interference: {'⚠️ Detected' if result.interference_detected else '✅ None'}")
    
    if spoofing_indicators and spoofing_suspicious:
        print(f"   └─ Indicators: {', '.join(spoofing_indicators)}")
    
    # === GENERATE PROPER SUMMARY BASED ON DISPLAYED SCORE ===
    # Don't use result.summary - it's based on internal score with different weights
    overall_score = qs.overall
    
    if overall_score >= 90:
        quality_text = "Excellent GNSS signal quality"
    elif overall_score >= 80:
        quality_text = "Good GNSS signal quality"
    elif overall_score >= 70:
        quality_text = "Fair GNSS signal quality"
    elif overall_score >= 60:
        quality_text = "Degraded GNSS signal quality"
    else:
        quality_text = "Poor GNSS signal quality"
    
    # Add warnings if needed
    warnings = []
    if jamming_detected:
        warnings.append("jamming detected")
    if spoofing_suspicious:
        warnings.append("spoofing indicators present")
    if result.interference_detected:
        warnings.append("interference events detected")
    if lock_integrity_score < 50:
        warnings.append("significant lock loss issues")
    
    if warnings:
        summary_text = f"{quality_text} - WARNING: {', '.join(warnings)}"
    else:
        summary_text = quality_text
    
    print(f"\n📝 {summary_text}")
    
    # === ANALYSIS CONCLUSION ===
    print(f"\n" + "=" * 70)
    print(f"📋 CONCLUSION:")
    print(f"=" * 70)
    
    conclusions = []
    
    # Overall assessment
    if overall_score >= 90:
        conclusions.append("✅ Data quality is EXCELLENT - suitable for high-precision applications (PPP/PPK)")
    elif overall_score >= 80:
        conclusions.append("✅ Data quality is GOOD - suitable for standard GNSS applications")
    elif overall_score >= 70:
        conclusions.append("⚠️ Data quality is FAIR - usable but may have reduced accuracy")
    elif overall_score >= 60:
        conclusions.append("⚠️ Data quality is DEGRADED - review anomalies before use")
    else:
        conclusions.append("❌ Data quality is POOR - significant issues detected")
    
    # Signal strength assessment
    if result.mean_cn0 >= 45:
        conclusions.append(f"✅ Signal strength EXCELLENT ({result.mean_cn0:.1f} dB-Hz average)")
    elif result.mean_cn0 >= 40:
        conclusions.append(f"✅ Signal strength GOOD ({result.mean_cn0:.1f} dB-Hz average)")
    elif result.mean_cn0 >= 35:
        conclusions.append(f"⚠️ Signal strength MODERATE ({result.mean_cn0:.1f} dB-Hz average)")
    else:
        conclusions.append(f"❌ Signal strength LOW ({result.mean_cn0:.1f} dB-Hz) - possible interference")
    
    # Lock integrity
    if lock_integrity_score >= 80:
        conclusions.append("✅ Signal continuity is excellent (minimal lock losses)")
    elif lock_integrity_score >= 60:
        conclusions.append("✅ Signal continuity is acceptable")
    else:
        conclusions.append(f"⚠️ Signal continuity issues detected ({total_data_gaps} data gaps)")
    
    # Threat summary
    if not (jamming_detected or spoofing_suspicious or result.interference_detected):
        conclusions.append("✅ No significant threats detected")
    else:
        if jamming_detected:
            conclusions.append("🚨 JAMMING DETECTED - data may be compromised")
        if spoofing_suspicious:
            conclusions.append("🚨 SPOOFING INDICATORS - verify data integrity")
        if result.interference_detected:
            conclusions.append("⚠️ Interference events detected - review anomaly timeline")
    
    # Post-processing recommendation
    if overall_score >= 70 and result.mean_cn0 >= 35 and lock_integrity_score >= 50:
        conclusions.append("✅ Data suitable for post-processing (PPP/RTK)")
    else:
        conclusions.append("⚠️ Review issues before post-processing")
    
    for c in conclusions:
        print(f"   {c}")
    
    print(f"\n" + "=" * 70)
    
    # Overall assessment
    add_report("\n" + "-"*70)
    score = result.quality_score.overall
    if score >= 80:
        add_report("📈 ASSESSMENT: EXCELLENT - High quality data for precise positioning")
    elif score >= 60:
        add_report("📈 ASSESSMENT: GOOD - Suitable for standard GNSS applications")
    elif score >= 40:
        add_report("📈 ASSESSMENT: DEGRADED - Some issues detected, review anomalies")
    else:
        add_report("📈 ASSESSMENT: POOR - Significant interference or equipment issues")
    
    # Store for export
    analysis_results['figures'] = figures
    lock_integrity_data = {
        'score': lock_integrity_score,
        'total_cycle_slips': total_cycle_slips,
        'total_data_gaps': total_data_gaps,
        'slips_per_hour': slips_per_hour,
    }
    analysis_results['lock_integrity'] = lock_integrity_data
    analysis_results['report_html'] = generate_html_report(result, const_data, anomalies, lock_integrity_data)
    
    print(f"\n✅ Generated {len(figures)} interactive plots")


def clear_all(btn):
    """Clear all data and outputs"""
    loaded_data.update({
        'obs_content': None, 'obs_filename': None, 'obs_path': None,
        'nav_content': None, 'nav_filename': None, 'nav_path': None,
    })
    analysis_results['data'] = None
    analysis_results['figures'] = {}
    
    with info_out:
        clear_output()
    with results_out:
        clear_output()
    
    progress.value = 0
    progress.layout.visibility = 'hidden'
    export_btn.disabled = True
    status.value = "<b>Status:</b> Cleared"


def export_results(btn):
    """Export results as HTML report"""
    if not analysis_results['data']:
        with info_out:
            print("❌ No analysis results to export!")
        return
    
    with info_out:
        clear_output()
        print("📦 Preparing export...")
        
        import base64
        
        result = analysis_results['data']
        html = analysis_results.get('report_html', '')
        
        if html:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"cn0_report_{timestamp}.html"
            
            b64 = base64.b64encode(html.encode()).decode()
            
            print(f"✅ Report ready: {filename}")
            
            display(HTML(f'''
            <div style="margin: 20px 0; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 10px; text-align: center;">
                <a download="{filename}" href="data:text/html;base64,{b64}" 
                   style="display:inline-block;padding:15px 30px;background:white;
                          color:#667eea;text-decoration:none;border-radius:8px;font-weight:bold;font-size:16px;">
                   📥 Download HTML Report ({len(html)//1024} KB)
                </a>
            </div>
            '''))


# Connect handlers
load_btn.on_click(load_files)
analyze_btn.on_click(run_analysis)
clear_btn.on_click(clear_all)
export_btn.on_click(export_results)

# ============ LAYOUT ============
layout = widgets.VBox([
    header,
    widgets.HTML("<hr>"),
    
    # Observation file section
    obs_section,
    widgets.HBox([obs_upload, widgets.HTML("&nbsp; OR &nbsp;"), obs_path_input]),
    
    widgets.HTML("<br>"),
    
    # Navigation file section  
    nav_section,
    widgets.HBox([nav_upload, widgets.HTML("&nbsp; OR &nbsp;"), nav_path_input]),
    auto_download_nav,
    
    widgets.HTML("<hr>"),
    load_btn,
    
    widgets.HTML("<hr>"),
    
    # Configuration
    config_section,
    preset_dropdown,
    preset_help,
    elevation_slider,
    time_bin_slider,
    widgets.HBox(list(system_checks.values())),
    
    widgets.HTML("<hr>"),
    
    # Action buttons
    widgets.HBox([analyze_btn, export_btn, clear_btn]),
    progress,
    status,
    
    widgets.HTML("<hr>"),
    info_out,
    results_out,
])

display(layout)
print("✅ CN0 Widget Ready with Presets")
print("   Presets: Full | Quick | Interference | Jamming | Spoofing")
print("   Features: Lock Integrity, Fixed Anomaly Graph, Research-based Thresholds")

✅ CN0 Widget Ready with Presets
   Presets: Full | Quick | Interference | Jamming | Spoofing
   Features: Lock Integrity, Fixed Anomaly Graph, Research-based Thresholds


# Separated output - Modified Widget

In [1]:
# =============================================================================
# CN0 Analysis Widget - MODIFIED VERSION with Separate Output Buttons
# =============================================================================
# Replace your existing widget cell (Cell 21) with this code
#
# CHANGES:
# - Removed single "Run Analysis" button
# - Added 5 separate buttons for different outputs:
#   1. 📊 Summary & Score - Text info + Quality radar chart
#   2. 🗺️ Heatmap - CN0 heatmap by azimuth/elevation AND time vs satellite
#   3. 📈 SNR Graphs - CN0 timeseries plots
#   4. 🛰️ Skyplot - Satellite skyplot
#   5. ⚠️ Anomalies - Anomaly timeline
# - Export runs full analysis silently and offers download
# =============================================================================

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
import gzip
import tempfile
from pathlib import Path
from datetime import datetime, timedelta
import urllib.request
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import geoveil_cn0 as gcn0

# Storage for loaded data
loaded_data = {
    'obs_content': None,
    'obs_filename': None,
    'obs_path': None,
    'nav_content': None,
    'nav_filename': None,
    'nav_path': None,
}

# Store analysis results
analysis_results = {'data': None, 'figures': {}, 'report_html': ''}

# ============ NAVIGATION DOWNLOADER ============
class NavDownloader:
    """Minimal nav downloader for notebook use"""
    
    IGS_SERVERS = [
        ('igs.bkg.bund.de', '/root_ftp/IGS/BRDC/{year}/{doy:03d}/{filename}', 'BKG'),
        ('igs.ign.fr', '/pub/igs/data/{year}/{doy:03d}/{filename}', 'IGN'),
    ]
    
    @staticmethod
    def parse_rinex_date(filename):
        """Extract year, doy from RINEX filename"""
        import re
        try:
            # RINEX 3/4 format: SSSSMMMMR_U_YYYYDDDHHMM_...
            parts = [p for p in filename.split('_') if p]
            if len(parts) >= 4 and len(parts[2]) >= 7:
                ts = parts[2]
                return int(ts[0:4]), int(ts[4:7])
            
            # RINEX 2 standard format: ssssdddf.yyt (e.g., bucu1520.25o)
            # where ssss=station, ddd=doy, f=file seq, yy=year, t=type
            match = re.match(r'^[a-zA-Z0-9]{4}(\d{3})\d?\.(\d{2})[oOnNmMgG]$', filename)
            if match:
                doy = int(match.group(1))
                yr = int(match.group(2))
                year = 2000 + yr if yr < 80 else 1900 + yr
                return year, doy
            
            # Extended RINEX 2 format: ssssdddsss.yyo (e.g., 0000152157.25o)
            # where ssss=station, ddd=doy, sss=sequence/extra, yy=year
            match = re.match(r'^[a-zA-Z0-9]{4}(\d{3})\d*\.(\d{2})[oOnNmMgG]$', filename)
            if match:
                doy = int(match.group(1))
                yr = int(match.group(2))
                year = 2000 + yr if yr < 80 else 1900 + yr
                return year, doy
            
            # Try to find any 3-digit DOY pattern after 4-char station
            match = re.match(r'^.{4}(\d{3}).*\.(\d{2})[oOnNgGmM]', filename)
            if match:
                doy = int(match.group(1))
                yr = int(match.group(2))
                year = 2000 + yr if yr < 80 else 1900 + yr
                if 1 <= doy <= 366:  # Valid DOY range
                    return year, doy
                
        except Exception as e:
            print(f"   Date parse error: {e}")
        return None, None
    
    @staticmethod
    def parse_rinex_header(content):
        """Extract year, doy from RINEX file content header (more reliable)"""
        from datetime import date
        try:
            # Handle bytes
            if isinstance(content, bytes):
                content = content.decode('utf-8', errors='ignore')
            
            for line in content.split('\n'):
                if 'TIME OF FIRST OBS' in line:
                    parts = line.split()
                    if len(parts) >= 6:
                        year = int(float(parts[0]))
                        month = int(float(parts[1]))
                        day = int(float(parts[2]))
                        doy = date(year, month, day).timetuple().tm_yday
                        return year, doy
                if 'END OF HEADER' in line:
                    break
        except Exception as e:
            print(f"   Header parse error: {e}")
        return None, None
    
    @staticmethod
    def download(year, doy, output_dir, log_func=print):
        """Download BRDC navigation file"""
        filename = f"BRDC00IGS_R_{year}{doy:03d}0000_01D_MN.rnx.gz"
        out_path = Path(output_dir) / filename.replace('.gz', '')
        
        if out_path.exists():
            log_func(f"✓ Nav file exists: {out_path.name}")
            return out_path
        
        for host, path_tpl, name in NavDownloader.IGS_SERVERS:
            try:
                url_path = path_tpl.format(year=year, doy=doy, filename=filename)
                url = f"https://{host}{url_path}"
                log_func(f"  Trying {name}...")
                
                req = urllib.request.Request(url)
                req.add_header('User-Agent', 'Mozilla/5.0 GNSS-Notebook')
                
                with urllib.request.urlopen(req, timeout=60) as resp:
                    data = resp.read()
                
                decompressed = gzip.decompress(data)
                with open(out_path, 'wb') as f:
                    f.write(decompressed)
                
                log_func(f"✓ Downloaded: {out_path.name} ({len(decompressed)/1024:.1f} KB)")
                return out_path
                
            except Exception as e:
                log_func(f"  ✗ {name}: {e}")
                continue
        
        return None

# ============ FILE INPUT WIDGETS ============

header = widgets.HTML("<h3>📡 GeoVeil CN0 Analysis - GNSS Signal Quality</h3>")

# === OBSERVATION FILE ===
obs_section = widgets.HTML("<b>Observation File</b> (required)")

obs_upload = widgets.FileUpload(
    accept='.obs,.rnx,.crx,.24o,.23o,.22o,.21o,.20o,.25o,.gz,.Z,*',
    multiple=False,
    description='Upload OBS',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

obs_path_input = widgets.Text(
    value='',
    placeholder='/path/to/observation.rnx',
    description='OBS Path:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='450px')
)

# === NAVIGATION FILE ===
nav_section = widgets.HTML("<b>Navigation/Ephemeris</b> (for elevation & skyplots)")

nav_upload = widgets.FileUpload(
    accept='.nav,.rnx,.24n,.24g,.25n,.sp3,.SP3,.gz,.Z,*',
    multiple=False,
    description='Upload NAV',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

nav_path_input = widgets.Text(
    value='',
    placeholder='/path/to/navigation.rnx or .sp3',
    description='NAV Path:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='450px')
)

auto_download_nav = widgets.Checkbox(
    value=True,
    description='Auto-download BRDC if missing',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='250px')
)

load_btn = widgets.Button(
    description='📥 Load Files',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

# === ANALYSIS CONFIG ===
config_section = widgets.HTML("<b>Analysis Configuration</b>")

elevation_slider = widgets.FloatSlider(
    value=5.0, min=0.0, max=30.0, step=1.0,
    description='Elevation Cutoff (°):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

time_bin_slider = widgets.IntSlider(
    value=60, min=10, max=300, step=10,
    description='Time Bin (sec):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

system_checks = {
    'G': widgets.Checkbox(value=True, description='GPS', layout=widgets.Layout(width='100px')),
    'R': widgets.Checkbox(value=True, description='GLONASS', layout=widgets.Layout(width='100px')),
    'E': widgets.Checkbox(value=True, description='Galileo', layout=widgets.Layout(width='100px')),
    'C': widgets.Checkbox(value=True, description='BeiDou', layout=widgets.Layout(width='100px')),
}

# === OUTPUT BUTTONS (NEW!) ===
buttons_section = widgets.HTML("<b>📊 Analysis Outputs</b> <i>(click to generate each view)</i>")

btn_summary = widgets.Button(
    description='📊 Summary & Score',
    button_style='primary',
    layout=widgets.Layout(width='160px', height='35px'),
    tooltip='Show text summary and quality radar chart'
)

btn_heatmap = widgets.Button(
    description='🗺️ Heatmaps',
    button_style='info',
    layout=widgets.Layout(width='120px', height='35px'),
    tooltip='Show CN0 heatmaps (Az/El + Time vs Satellite)'
)

btn_snr = widgets.Button(
    description='📈 SNR Graphs',
    button_style='info',
    layout=widgets.Layout(width='130px', height='35px'),
    tooltip='Show CN0 timeseries plots'
)

btn_skyplot = widgets.Button(
    description='🛰️ Skyplot',
    button_style='info',
    layout=widgets.Layout(width='120px', height='35px'),
    tooltip='Show satellite skyplot'
)

btn_anomaly = widgets.Button(
    description='⚠️ Anomalies',
    button_style='warning',
    layout=widgets.Layout(width='130px', height='35px'),
    tooltip='Show anomaly timeline'
)

# === EXPORT BUTTON ===
export_btn = widgets.Button(
    description='📥 Export Full Report',
    button_style='success',
    layout=widgets.Layout(width='180px', height='35px'),
    tooltip='Run complete analysis and download HTML report'
)

clear_btn = widgets.Button(
    description='🗑️ Clear',
    button_style='danger',
    layout=widgets.Layout(width='80px', height='35px')
)

# Progress & Status
progress = widgets.FloatProgress(
    value=0, min=0, max=1.0,
    description='Progress:',
    layout=widgets.Layout(width='400px', visibility='hidden')
)

status = widgets.HTML(value="<b>Status:</b> Ready - load files to begin")

# Output areas
info_out = widgets.Output()
results_out = widgets.Output()

# ============ CORE ANALYSIS FUNCTION ============

def run_core_analysis(silent=False):
    """Run the CN0 analysis - returns result or None"""
    global analysis_results
    
    if not loaded_data['obs_content']:
        if not silent:
            with results_out:
                clear_output()
                print("❌ No observation file loaded!")
        return None
    
    try:
        import geoveil_cn0 as gcn0
        import tempfile
        
        # Get enabled systems
        systems = [s for s, cb in system_checks.items() if cb.value]
        
        # Create config
        config = gcn0.AnalysisConfig(
            min_elevation=elevation_slider.value,
            time_bin_seconds=time_bin_slider.value,
            systems=systems,
            detect_anomalies=True,
        )
        
        # Create analyzer
        analyzer = gcn0.CN0Analyzer(config)
        
        # Save content to temp files (analyzer expects file paths)
        temp_dir = tempfile.gettempdir()
        
        obs_path = os.path.join(temp_dir, loaded_data['obs_filename'])
        with open(obs_path, 'wb') as f:
            f.write(loaded_data['obs_content'])
        
        nav_path = None
        if loaded_data['nav_content']:
            nav_path = os.path.join(temp_dir, loaded_data['nav_filename'])
            with open(nav_path, 'wb') as f:
                f.write(loaded_data['nav_content'])
        
        # Run analysis
        if nav_path:
            result = analyzer.analyze_with_nav(obs_path, nav_path)
        else:
            result = analyzer.analyze_file(obs_path)
        
        analysis_results['data'] = result
        return result
        
    except Exception as e:
        if not silent:
            with results_out:
                clear_output()
                print(f"❌ Analysis error: {e}")
        return None

# ============ OUTPUT HANDLERS ============

def show_summary(btn):
    """Show text summary and quality score radar"""
    with results_out:
        clear_output()
        print("📊 Generating summary...")
    
    result = analysis_results.get('data')
    if not result:
        result = run_core_analysis()
    
    if not result:
        return
    
    with results_out:
        clear_output()
        
        # === TEXT SUMMARY ===
        qs = result.quality_score
        
        print("=" * 70)
        print("📊 GEOVEIL CN0 ANALYSIS RESULTS")
        print("=" * 70)
        
        print(f"\n📁 File:")
        print(f"   RINEX Version: {result.rinex_version}")
        print(f"   Duration: {result.duration_hours:.2f} hours ({result.epoch_count} epochs)")
        print(f"   Station: {result.station_name or 'Unknown'}")
        print(f"   Constellations: {', '.join(result.get_systems())}")
        
        # === CALCULATE LOCK LOSS METRICS ===
        total_cycle_slips = 0
        total_data_gaps = 0
        total_satellites = 0
        
        for sys_name in ['GPS', 'GLONASS', 'Galileo', 'BeiDou']:
            stats = result.get_constellation_summary(sys_name)
            if stats:
                total_cycle_slips += int(stats.get('cycle_slips', 0))
                total_data_gaps += int(stats.get('data_gaps', 0))
                total_satellites += int(stats.get('satellite_count', 0))
        
        # Calculate rates
        duration_hours = max(result.duration_hours, 0.01)
        slips_per_hour = total_cycle_slips / duration_hours
        gaps_per_hour = total_data_gaps / duration_hours
        
        # Lock Loss Score (0-100, higher is better = fewer losses)
        # Based on cycle slips and data gaps per satellite per hour
        if total_satellites > 0:
            slips_per_sat_hour = slips_per_hour / total_satellites
            gaps_per_sat_hour = gaps_per_hour / total_satellites
            
            # Score calculation: penalize for slips and gaps
            # Target: <0.1 slips/sat/hour = 100, >2 slips/sat/hour = 0
            slip_score = max(0, min(100, 100 - (slips_per_sat_hour * 50)))
            gap_score = max(0, min(100, 100 - (gaps_per_sat_hour * 25)))
            lock_integrity_score = (slip_score * 0.6 + gap_score * 0.4)
        else:
            lock_integrity_score = 0
            slips_per_sat_hour = 0
            gaps_per_sat_hour = 0
        
        print(f"\n🏆 QUALITY SCORE: {qs.overall:.0f}/100 ({qs.rating})")
        print(f"   CN0 Quality:   {qs.cn0_quality:.0f}")
        print(f"   Availability:  {qs.availability:.0f}")
        print(f"   Continuity:    {qs.continuity:.0f}")
        print(f"   Stability:     {qs.stability:.0f}")
        print(f"   Diversity:     {qs.diversity:.0f}")
        print(f"   Lock Integrity: {lock_integrity_score:.0f}")
        print(f"   Post-processing suitable: {'✅ Yes' if qs.overall >= 70 else '❌ No'}")
        
        print(f"\n📶 SIGNAL QUALITY:")
        print(f"   Average CN0: {result.mean_cn0:.1f} dB-Hz")
        print(f"   Std Dev: {result.cn0_std_dev:.1f} dB-Hz")
        print(f"   Range: {result.min_cn0:.1f} - {result.max_cn0:.1f} dB-Hz")
        
        # === LOCK LOSS SECTION ===
        print(f"\n🔓 LOCK LOSS / SIGNAL CONTINUITY:")
        print(f"   Total Cycle Slips: {total_cycle_slips} ({slips_per_hour:.1f}/hour)")
        print(f"   Total Data Gaps:   {total_data_gaps} ({gaps_per_hour:.1f}/hour)")
        print(f"   Per-Satellite Rate: {slips_per_sat_hour:.2f} slips/sat/hour")
        
        # Lock loss assessment
        if lock_integrity_score >= 90:
            lock_status = "✅ Excellent - Minimal signal interruptions"
        elif lock_integrity_score >= 70:
            lock_status = "✅ Good - Acceptable continuity"
        elif lock_integrity_score >= 50:
            lock_status = "⚠️ Moderate - Some signal interruptions"
        elif lock_integrity_score >= 30:
            lock_status = "⚠️ Poor - Frequent lock losses"
        else:
            lock_status = "🚨 Critical - Severe continuity issues"
        print(f"   Lock Integrity Score: {lock_integrity_score:.0f}/100 - {lock_status}")
        
        print(f"\n🛡️ THREAT ASSESSMENT:")
        
        # === IMPROVED THREAT DETECTION ===
        # Don't blindly trust the library's spoofing flag - it has false positives
        # When ephemeris doesn't cover all satellites (especially BeiDou), it triggers falsely
        
        # Jamming: Only if low CN0 + library flag
        jamming_detected = result.jamming_detected and result.mean_cn0 < 35.0
        
        # Spoofing: Check per-constellation std to avoid false positives from incomplete ephemeris
        constellation_stds = []
        for sys_name in ['GPS', 'GLONASS', 'Galileo', 'BeiDou']:
            stats = result.get_constellation_summary(sys_name)
            if stats:
                try:
                    std_cn0 = float(stats.get('std_cn0', stats.get('cn0_std', 5.0)))
                    constellation_stds.append(std_cn0)
                except:
                    pass
        
        avg_constellation_std = sum(constellation_stds) / len(constellation_stds) if constellation_stds else 5.0
        
        # Real spoofing indicators:
        # 1. Very low CN0 std across constellations (< 2 dB) - signals too uniform
        # 2. AND elevated average CN0 (> 50 dB-Hz) - spoofer typically overpowers
        # 3. AND overall std also low
        spoofing_indicators = []
        if avg_constellation_std < 2.0:
            spoofing_indicators.append("CN0 uniformity suspiciously low")
        if result.mean_cn0 > 50.0:
            spoofing_indicators.append("CN0 elevated (possible high-power signal)")
        if result.cn0_std_dev < 1.0:
            spoofing_indicators.append("Overall signal variance very low")
        
        # Only flag spoofing if multiple indicators present
        spoofing_suspicious = len(spoofing_indicators) >= 2
        
        # If library says spoofing but our checks don't agree, it's likely false positive
        if result.spoofing_detected and not spoofing_suspicious:
            print(f"   Jamming:      {'🚨 DETECTED' if jamming_detected else '✅ None'}")
            print(f"   Spoofing:     ⚠️ Flag raised (likely false positive - incomplete ephemeris)")
            print(f"   Interference: {'⚠️ Detected' if result.interference_detected else '✅ None'}")
        else:
            print(f"   Jamming:      {'🚨 DETECTED' if jamming_detected else '✅ None'}")
            print(f"   Spoofing:     {'🚨 DETECTED' if spoofing_suspicious else '✅ None'}")
            print(f"   Interference: {'⚠️ Detected' if result.interference_detected else '✅ None'}")
        
        if spoofing_indicators and spoofing_suspicious:
            print(f"   └─ Indicators: {', '.join(spoofing_indicators)}")
        
        # === GENERATE PROPER SUMMARY BASED ON DISPLAYED SCORE ===
        # Don't use result.summary - it's based on internal score with different weights
        overall_score = qs.overall
        
        if overall_score >= 90:
            quality_text = "Excellent GNSS signal quality"
        elif overall_score >= 80:
            quality_text = "Good GNSS signal quality"
        elif overall_score >= 70:
            quality_text = "Fair GNSS signal quality"
        elif overall_score >= 60:
            quality_text = "Degraded GNSS signal quality"
        else:
            quality_text = "Poor GNSS signal quality"
        
        # Add warnings if needed
        warnings = []
        if jamming_detected:
            warnings.append("jamming detected")
        if spoofing_suspicious:
            warnings.append("spoofing indicators present")
        if result.interference_detected:
            warnings.append("interference events detected")
        if lock_integrity_score < 50:
            warnings.append("significant lock loss issues")
        
        if warnings:
            summary_text = f"{quality_text} - WARNING: {', '.join(warnings)}"
        else:
            summary_text = quality_text
        
        print(f"\n📝 {summary_text}")
        
        # === ANALYSIS CONCLUSION ===
        print(f"\n" + "=" * 70)
        print(f"📋 CONCLUSION:")
        print(f"=" * 70)
        
        conclusions = []
        
        # Overall assessment
        if overall_score >= 90:
            conclusions.append("✅ Data quality is EXCELLENT - suitable for high-precision applications (PPP/PPK)")
        elif overall_score >= 80:
            conclusions.append("✅ Data quality is GOOD - suitable for standard GNSS applications")
        elif overall_score >= 70:
            conclusions.append("⚠️ Data quality is FAIR - usable but may have reduced accuracy")
        elif overall_score >= 60:
            conclusions.append("⚠️ Data quality is DEGRADED - review anomalies before use")
        else:
            conclusions.append("❌ Data quality is POOR - significant issues detected")
        
        # Signal strength assessment
        if result.mean_cn0 >= 45:
            conclusions.append(f"✅ Signal strength EXCELLENT ({result.mean_cn0:.1f} dB-Hz average)")
        elif result.mean_cn0 >= 40:
            conclusions.append(f"✅ Signal strength GOOD ({result.mean_cn0:.1f} dB-Hz average)")
        elif result.mean_cn0 >= 35:
            conclusions.append(f"⚠️ Signal strength MODERATE ({result.mean_cn0:.1f} dB-Hz average)")
        else:
            conclusions.append(f"❌ Signal strength LOW ({result.mean_cn0:.1f} dB-Hz) - possible interference")
        
        # Lock integrity
        if lock_integrity_score >= 80:
            conclusions.append("✅ Signal continuity is excellent (minimal lock losses)")
        elif lock_integrity_score >= 60:
            conclusions.append("✅ Signal continuity is acceptable")
        else:
            conclusions.append(f"⚠️ Signal continuity issues detected ({total_data_gaps} data gaps)")
        
        # Threat summary
        if not (jamming_detected or spoofing_suspicious or result.interference_detected):
            conclusions.append("✅ No significant threats detected")
        else:
            if jamming_detected:
                conclusions.append("🚨 JAMMING DETECTED - data may be compromised")
            if spoofing_suspicious:
                conclusions.append("🚨 SPOOFING INDICATORS - verify data integrity")
            if result.interference_detected:
                conclusions.append("⚠️ Interference events detected - review anomaly timeline")
        
        # Post-processing recommendation
        if overall_score >= 70 and result.mean_cn0 >= 35 and lock_integrity_score >= 50:
            conclusions.append("✅ Data suitable for post-processing (PPP/RTK)")
        else:
            conclusions.append("⚠️ Review issues before post-processing")
        
        for c in conclusions:
            print(f"   {c}")
        
        print(f"\n" + "=" * 70)
        
        # === CONSTELLATION SUMMARY ===
        print(f"\n🛰️ CONSTELLATION SUMMARY:")
        for sys_name in ['GPS', 'GLONASS', 'Galileo', 'BeiDou']:
            stats = result.get_constellation_summary(sys_name)
            if stats:
                sat_count = stats.get('satellite_count', '0')
                expected = stats.get('satellites_expected', sat_count)
                mean_cn0 = stats.get('mean_cn0', '0')
                std_cn0 = stats.get('std_cn0', '0')
                cycle_slips = stats.get('cycle_slips', '0')
                data_gaps = stats.get('data_gaps', '0')
                
                pct = int(sat_count) / max(int(expected), 1) * 100
                slips_rate = int(cycle_slips) / duration_hours
                print(f"\n   {sys_name}:")
                print(f"      Satellites: {sat_count}/{expected} ({pct:.0f}%)")
                print(f"      CN0: {float(mean_cn0):.1f} ± {float(std_cn0):.2f} dB-Hz")
                print(f"      Cycle Slips: {cycle_slips} ({slips_rate:.1f}/hour)")
                print(f"      Data Gaps: {data_gaps}")
        
        print("\n" + "=" * 70)
        
        # === QUALITY RADAR CHART (now includes Lock Integrity) ===
        print("\n📈 Quality Score Radar:")
        
        fig = go.Figure()
        
        # Add Lock Integrity to the radar chart
        categories = ['Availability', 'CN0 Quality', 'Stability', 'Diversity', 'Continuity', 'Lock Integrity']
        values = [qs.availability, qs.cn0_quality, qs.stability, qs.diversity, qs.continuity, lock_integrity_score]
        values.append(values[0])  # Close the polygon
        
        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories + [categories[0]],
            fill='toself',
            fillcolor='rgba(99, 110, 250, 0.3)',
            line=dict(color='rgb(99, 110, 250)', width=2),
            name='Quality'
        ))
        
        # Rating color
        color = '#22c55e' if qs.overall >= 80 else '#eab308' if qs.overall >= 60 else '#ef4444'
        
        fig.update_layout(
            polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
            showlegend=False,
            title=f"Quality Score: {qs.overall:.0f}/100 ({qs.rating})",
            height=550,
            width=650
        )
        
        fig.show()


def show_heatmap(btn):
    """Show BOTH CN0 heatmaps: Az/El and Time vs Satellite"""
    with results_out:
        clear_output()
        print("🗺️ Generating heatmaps...")
    
    result = analysis_results.get('data')
    if not result:
        result = run_core_analysis()
    
    if not result:
        return
    
    with results_out:
        clear_output()
        
        # =====================================================================
        # HEATMAP 1: Time vs Satellite PRN (C/N₀ Heatmap)
        # =====================================================================
        print("🔥 C/N₀ Heatmap - Time vs Satellite")
        
        timestamps = result.get_timestamps()
        
        # Get satellite timeseries from JSON (most reliable method from original widget)
        try:
            result_json = json.loads(result.to_json())
            sat_timeseries = result_json.get('timeseries', {}).get('satellite_timeseries', {})
            
            if sat_timeseries and len(sat_timeseries) > 0:
                print(f"   Found {len(sat_timeseries)} satellites in timeseries")
                
                # Get all timestamps from the data
                all_times = set()
                for sat_id, sat_data in sat_timeseries.items():
                    if isinstance(sat_data, dict):
                        series = sat_data.get('cn0_series', sat_data.get('series', []))
                        if isinstance(series, list):
                            for point in series:
                                if isinstance(point, dict):
                                    all_times.add(point.get('timestamp', point.get('time', '')))
                
                all_times = sorted([t for t in all_times if t])
                
                if all_times:
                    # Subsample if too many points
                    max_time_points = 500
                    if len(all_times) > max_time_points:
                        step = len(all_times) // max_time_points
                        all_times = all_times[::step]
                    
                    # Sort satellites by constellation then PRN
                    def sat_sort_key(s):
                        if len(s) >= 2:
                            sys = s[0]
                            try:
                                prn = int(s[1:])
                            except:
                                prn = 0
                            sys_order = {'G': 0, 'R': 1, 'E': 2, 'C': 3, 'J': 4, 'I': 5, 'S': 6}
                            return (sys_order.get(sys, 9), prn)
                        return (9, 0)
                    
                    all_satellites = sorted(sat_timeseries.keys(), key=sat_sort_key, reverse=True)
                    
                    # Build z matrix (satellites x time)
                    z_matrix = []
                    sat_labels = []
                    
                    for sat in all_satellites:
                        sat_data = sat_timeseries.get(sat, {})
                        
                        if isinstance(sat_data, dict):
                            cn0_series = sat_data.get('cn0_series', sat_data.get('series', []))
                        else:
                            continue
                        
                        # Create lookup dict
                        cn0_by_time = {}
                        if isinstance(cn0_series, list):
                            for p in cn0_series:
                                if isinstance(p, dict):
                                    t = p.get('timestamp', p.get('time', ''))
                                    v = p.get('value', p.get('cn0', None))
                                    if t and v is not None:
                                        cn0_by_time[t] = v
                        
                        # Fill row
                        row = []
                        for t in all_times:
                            val = cn0_by_time.get(t, None)
                            row.append(val if val is not None else None)
                        
                        # Only add if we have some data
                        valid_count = sum(1 for v in row if v is not None)
                        if valid_count > len(all_times) * 0.05:  # At least 5% data
                            z_matrix.append(row)
                            sat_labels.append(sat)
                    
                    if z_matrix:
                        # Parse timestamps for display
                        time_labels = pd.to_datetime(all_times)
                        
                        fig1 = go.Figure(go.Heatmap(
                            z=z_matrix,
                            x=time_labels,
                            y=sat_labels,
                            colorscale='Viridis',
                            colorbar=dict(title='C/N₀<br>(dB-Hz)'),
                            hoverongaps=False,
                            hovertemplate='Satellite: %{y}<br>Time: %{x}<br>CN0: %{z:.1f} dB-Hz<extra></extra>',
                            zmin=25,
                            zmax=55
                        ))
                        
                        fig1.update_layout(
                            title='🔥 C/N₀ Heatmap - Time vs Satellite',
                            xaxis_title='Time (UTC)',
                            yaxis_title='Satellite PRN',
                            width=1100,
                            height=max(400, len(sat_labels) * 20 + 150),
                            xaxis=dict(type='date'),
                            yaxis=dict(tickmode='array', tickvals=sat_labels, ticktext=sat_labels)
                        )
                        
                        fig1.show()
                    else:
                        print("⚠️ No valid satellite data for heatmap")
                else:
                    print("⚠️ No timestamps found in satellite timeseries")
            else:
                print("⚠️ No satellite_timeseries data available")
        except Exception as e:
            print(f"⚠️ Could not create Time vs Satellite heatmap: {e}")
        
        # =====================================================================
        # HEATMAP 2: CN0 by Azimuth/Elevation
        # =====================================================================
        print("\n🗺️ CN0 Heatmap by Azimuth/Elevation")
        
        # Get skyplot data for azimuth/elevation heatmap
        skyplot_data = result.get_skyplot_data()
        
        if not skyplot_data:
            print("⚠️ No azimuth/elevation data available (need navigation file)")
            return
        
        # Build heatmap from satellite traces
        # Create bins for azimuth (0-360) and elevation (0-90)
        az_bins = list(range(0, 361, 15))  # 0, 15, 30, ..., 360
        el_bins = list(range(0, 91, 5))    # 0, 5, 10, ..., 90
        
        # Initialize grid
        cn0_sum = [[0.0 for _ in range(len(az_bins)-1)] for _ in range(len(el_bins)-1)]
        cn0_count = [[0 for _ in range(len(az_bins)-1)] for _ in range(len(el_bins)-1)]
        
        # Aggregate data from all satellites
        for sat_trace in skyplot_data:
            azimuths = [float(x) for x in sat_trace.get('azimuths', '').split(',') if x]
            elevations = [float(x) for x in sat_trace.get('elevations', '').split(',') if x]
            cn0_values = [float(x) for x in sat_trace.get('cn0_values', '').split(',') if x]
            
            for az, el, cn0 in zip(azimuths, elevations, cn0_values):
                az_idx = min(int(az / 15), len(az_bins) - 2)
                el_idx = min(int(el / 5), len(el_bins) - 2)
                cn0_sum[el_idx][az_idx] += cn0
                cn0_count[el_idx][az_idx] += 1
        
        # Compute mean CN0 for each bin
        cn0_grid = []
        for el_idx in range(len(el_bins) - 1):
            row = []
            for az_idx in range(len(az_bins) - 1):
                if cn0_count[el_idx][az_idx] > 0:
                    row.append(cn0_sum[el_idx][az_idx] / cn0_count[el_idx][az_idx])
                else:
                    row.append(None)  # No data
            cn0_grid.append(row)
        
        if all(all(v is None for v in row) for row in cn0_grid):
            print("⚠️ No heatmap data available (no valid observations)")
            return
        
        fig2 = go.Figure(go.Heatmap(
            z=cn0_grid,
            x=[f"{az_bins[i]}-{az_bins[i+1]}" for i in range(len(az_bins)-1)],
            y=[f"{el_bins[i]}-{el_bins[i+1]}" for i in range(len(el_bins)-1)],
            colorscale='Viridis',
            colorbar=dict(title='CN0 (dB-Hz)'),
            hoverongaps=False,
            zmin=30,
            zmax=55
        ))
        
        fig2.update_layout(
            title='CN0 Heatmap by Azimuth/Elevation',
            xaxis_title='Azimuth (°)',
            yaxis_title='Elevation (°)',
            width=850,
            height=500
        )
        
        fig2.show()


def show_snr_graphs(btn):
    """Show CN0 timeseries - Overall average + Per-constellation satellite graphs"""
    with results_out:
        clear_output()
        print("📈 Generating SNR timeseries...")
    
    result = analysis_results.get('data')
    if not result:
        result = run_core_analysis()
    
    if not result:
        return
    
    with results_out:
        clear_output()
        
        # Get timestamps
        timestamps = result.get_timestamps()
        
        if not timestamps:
            print("⚠️ No timeseries data available")
            return
        
        # Parse timestamps
        ts = pd.to_datetime(timestamps)
        
        # Get mean CN0 and satellite counts
        mean_cn0 = result.get_mean_cn0_series()
        sat_counts = result.get_satellite_count_series()
        
        if len(mean_cn0) == 0:
            print("⚠️ No CN0 timeseries data")
            return
        
        # =====================================================================
        # GRAPH 1: Overall Mean CN0 Timeseries with Satellite Count
        # =====================================================================
        print("📊 Overall CN0 Timeseries:")
        
        fig = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.08,
            row_heights=[0.7, 0.3],
            subplot_titles=('CN0 Timeseries', 'Satellite Count')
        )
        
        # Overall mean
        fig.add_trace(go.Scatter(
            x=ts, y=mean_cn0,
            name='Overall Mean',
            line=dict(color='black', width=2.5)
        ), row=1, col=1)
        
        # Satellite count
        fig.add_trace(go.Bar(
            x=ts, y=sat_counts,
            marker_color='#6b7280',
            showlegend=False
        ), row=2, col=1)
        
        # Warning threshold (Degraded at 35)
        fig.add_hline(y=35, line_dash='dash', line_color='orange', 
                      annotation_text='Degraded (35)', annotation_position='right', row=1, col=1)
        
        fig.update_layout(
            height=500,
            width=1100,
            title='CN0 Timeseries',
            legend=dict(orientation='h', y=1.12)
        )
        fig.update_yaxes(title_text='CN0 (dB-Hz)', row=1, col=1)
        fig.update_yaxes(title_text='Satellites', row=2, col=1)
        fig.update_xaxes(title_text='Time (UTC)', row=2, col=1)
        
        fig.show()
        
        # =====================================================================
        # GRAPH 2+: Per-Constellation Satellite CN0 Timeseries
        # =====================================================================
        
        # Distinct colors for satellites within each constellation
        sat_colors = [
            '#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A',
            '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52',
            '#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD',
            '#8C564B', '#E377C2', '#7F7F7F', '#BCBD22', '#17BECF'
        ]
        
        sys_to_const = {'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 'C': 'BeiDou'}
        
        # Get satellite timeseries from JSON
        try:
            result_json = json.loads(result.to_json())
            sat_timeseries = result_json.get('timeseries', {}).get('satellite_timeseries', {})
            
            if not sat_timeseries or len(sat_timeseries) == 0:
                print("\n⚠️ No per-satellite data available for constellation graphs")
                return
            
            # Organize by constellation
            const_satellites = {
                'GPS': [],
                'GLONASS': [],
                'Galileo': [],
                'BeiDou': []
            }
            
            for sat_id, sat_data in sat_timeseries.items():
                if len(sat_id) < 2:
                    continue
                
                system = sat_id[0]
                const_name = sys_to_const.get(system)
                if not const_name:
                    continue
                
                if isinstance(sat_data, dict):
                    cn0_series = sat_data.get('cn0_series', sat_data.get('series', []))
                else:
                    continue
                
                # Extract timestamps and values
                sat_timestamps = []
                sat_cn0_values = []
                
                if isinstance(cn0_series, list):
                    for p in cn0_series:
                        if isinstance(p, dict):
                            t = p.get('timestamp', p.get('time', ''))
                            v = p.get('value', p.get('cn0', None))
                            if t and v is not None:
                                sat_timestamps.append(t)
                                sat_cn0_values.append(v)
                
                if len(sat_cn0_values) > 0:
                    const_satellites[const_name].append({
                        'sat_id': sat_id,
                        'timestamps': pd.to_datetime(sat_timestamps),
                        'cn0_values': sat_cn0_values
                    })
            
            # Check if we have any data
            total_sats = sum(len(sats) for sats in const_satellites.values())
            if total_sats == 0:
                print("\n⚠️ No per-satellite data available for constellation graphs")
                return
            
            # Create a graph for each constellation that has data
            for const_name, satellites in const_satellites.items():
                if not satellites:
                    continue
                
                # Sort satellites by PRN number
                satellites.sort(key=lambda s: int(s['sat_id'][1:]) if s['sat_id'][1:].isdigit() else 0)
                
                print(f"\n📡 {const_name} C/N₀ Timeseries ({len(satellites)} satellites):")
                
                fig_const = go.Figure()
                
                # Add each satellite trace
                for i, sat_data in enumerate(satellites):
                    color = sat_colors[i % len(sat_colors)]
                    fig_const.add_trace(go.Scatter(
                        x=sat_data['timestamps'],
                        y=sat_data['cn0_values'],
                        name=sat_data['sat_id'],
                        mode='lines',
                        line=dict(width=1.5, color=color),
                        hovertemplate=f"{sat_data['sat_id']}<br>Time: %{{x}}<br>CN0: %{{y:.1f}} dB-Hz<extra></extra>"
                    ))
                
                # Add threshold lines
                fig_const.add_hline(y=35, line_dash='dash', line_color='orange', 
                                   annotation_text='Degraded (35)', annotation_position='right')
                fig_const.add_hline(y=25, line_dash='dash', line_color='red',
                                   annotation_text='Poor (25)', annotation_position='right')
                
                fig_const.update_layout(
                    title=f'📡 {const_name} - C/N₀ by Satellite',
                    xaxis_title='Time (UTC)',
                    yaxis_title='C/N₀ (dB-Hz)',
                    height=450,
                    width=1100,
                    yaxis=dict(range=[20, 60]),
                    legend=dict(
                        orientation='h',
                        yanchor='bottom',
                        y=1.02,
                        xanchor='right',
                        x=1,
                        font=dict(size=10)
                    ),
                    hovermode='x unified'
                )
                
                fig_const.show()
                
        except Exception as e:
            print(f"\n⚠️ Could not create per-constellation graphs: {e}")


def show_skyplot(btn):
    """Show satellite skyplot"""
    with results_out:
        clear_output()
        print("🛰️ Generating skyplot...")
    
    result = analysis_results.get('data')
    if not result:
        result = run_core_analysis()
    
    if not result:
        return
    
    with results_out:
        clear_output()
        
        # Get skyplot data (list of satellite traces)
        skyplot_data = result.get_skyplot_data()
        coverage = result.skyplot_coverage
        
        if not skyplot_data:
            print("⚠️ No skyplot data (need receiver position)")
            return
        
        fig = go.Figure()
        
        # Group by constellation
        const_colors = {'GPS': '#3b82f6', 'GLONASS': '#ef4444', 'Galileo': '#22c55e', 'BeiDou': '#f59e0b'}
        const_data = {}
        
        for sat_trace in skyplot_data:
            system = sat_trace.get('system', 'Other')
            const_name = {'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 'C': 'BeiDou'}.get(system, system)
            
            if const_name not in const_data:
                const_data[const_name] = {'r': [], 'theta': [], 'cn0': [], 'text': []}
            
            azimuths = [float(x) for x in sat_trace.get('azimuths', '').split(',') if x]
            elevations = [float(x) for x in sat_trace.get('elevations', '').split(',') if x]
            cn0_values = [float(x) for x in sat_trace.get('cn0_values', '').split(',') if x]
            sat_id = sat_trace.get('satellite', '')
            
            for az, el, cn0 in zip(azimuths, elevations, cn0_values):
                # Convert elevation to radius (90° at center, 0° at edge)
                r = 90 - el
                const_data[const_name]['r'].append(r)
                const_data[const_name]['theta'].append(az)
                const_data[const_name]['cn0'].append(cn0)
                const_data[const_name]['text'].append(f"{sat_id}: {cn0:.1f} dB-Hz")
        
        # Plot each constellation
        for i, (const_name, data) in enumerate(const_data.items()):
            if data['r']:
                fig.add_trace(go.Scatterpolar(
                    r=data['r'],
                    theta=data['theta'],
                    mode='markers',
                    name=const_name,
                    marker=dict(
                        size=8,
                        color=data['cn0'],
                        colorscale='Viridis',
                        cmin=25, cmax=55,
                        colorbar=dict(title='CN0 (dB-Hz)') if i == 0 else None,
                        showscale=(i == 0)
                    ),
                    text=data['text'],
                    hoverinfo='text'
                ))
        
        fig.update_layout(
            title=f'CN0 Skyplot (Coverage: {coverage:.1f}%)',
            width=650,
            height=650,
            polar=dict(
                radialaxis=dict(
                    range=[0, 90],
                    tickvals=[0, 30, 60, 90],
                    ticktext=['90°', '60°', '30°', '0°']
                ),
                angularaxis=dict(
                    tickvals=[0, 45, 90, 135, 180, 225, 270, 315],
                    ticktext=['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'],
                    direction='clockwise',
                    rotation=90
                )
            )
        )
        
        fig.show()


def show_anomalies(btn):
    """Show anomaly timeline"""
    with results_out:
        clear_output()
        print("⚠️ Generating anomaly timeline...")
    
    result = analysis_results.get('data')
    if not result:
        result = run_core_analysis()
    
    if not result:
        return
    
    with results_out:
        clear_output()
        
        anomalies = result.get_anomalies()
        
        if not anomalies:
            print("✅ No anomalies detected!")
            fig = go.Figure()
            fig.add_annotation(
                text='✅ No anomalies detected',
                xref='paper', yref='paper',
                x=0.5, y=0.5,
                showarrow=False,
                font=dict(size=24, color='green')
            )
            fig.update_layout(title='Anomaly Timeline', height=400)
            fig.show()
            return
        
        print(f"⚠️ Found {len(anomalies)} anomalies")
        
        # Parse anomalies
        parsed = []
        for a in anomalies:
            try:
                ts_str = a.get('start_time') or a.get('timestamp') or ''
                if ts_str:
                    parsed.append({
                        'time': pd.to_datetime(ts_str),
                        'severity': a.get('severity', 'low'),
                        'cn0_drop': float(a.get('cn0_drop', 0) or 0),
                        'type': a.get('anomaly_type', 'Unknown'),
                        'description': a.get('description', '')
                    })
            except:
                continue
        
        if not parsed:
            print("⚠️ Could not parse anomaly timestamps")
            return
        
        fig = go.Figure()
        
        severity_colors = {
            'critical': '#ef4444', 'Critical': '#ef4444',
            'high': '#f97316', 'High': '#f97316',
            'medium': '#eab308', 'Medium': '#eab308',
            'low': '#22c55e', 'Low': '#22c55e'
        }
        
        for severity in ['critical', 'high', 'medium', 'low']:
            sev_data = [d for d in parsed if d['severity'].lower() == severity]
            if sev_data:
                fig.add_trace(go.Scatter(
                    x=[d['time'] for d in sev_data],
                    y=[d['cn0_drop'] for d in sev_data],
                    mode='markers',
                    marker=dict(size=10, color=severity_colors.get(severity, '#888')),
                    name=f"{severity.capitalize()} ({len(sev_data)})",
                    text=[f"{d['type']}<br>{d['description']}" for d in sev_data],
                    hovertemplate="Time: %{x}<br>CN0 Drop: %{y:.1f} dB<br>%{text}<extra></extra>"
                ))
        
        fig.update_layout(
            title=f'Anomaly Timeline ({len(parsed)} events)',
            xaxis_title='Time (UTC)',
            xaxis=dict(type='date'),
            yaxis_title='CN0 Drop (dB)',
            height=450,
            legend=dict(orientation='h', y=1.1)
        )
        
        fig.show()


def export_report(btn):
    """Run full analysis silently and offer download"""
    with results_out:
        clear_output()
        print("📥 Generating full report (please wait)...")
        progress.layout.visibility = 'visible'
        progress.value = 0.1
    
    # Run analysis silently
    result = run_core_analysis(silent=True)
    
    if not result:
        with results_out:
            clear_output()
            print("❌ Could not generate report - check files are loaded")
        progress.layout.visibility = 'hidden'
        return
    
    progress.value = 0.5
    
    try:
        # Generate HTML report
        qs = result.quality_score
        anomalies = result.get_anomalies()
        
        # Generate all figures
        figures_html = ""
        
        # 1. Quality Radar
        fig_radar = go.Figure()
        categories = ['Availability', 'CN0 Quality', 'Stability', 'Diversity', 'Continuity']
        values = [qs.availability, qs.cn0_quality, qs.stability, qs.diversity, qs.continuity, qs.availability]
        fig_radar.add_trace(go.Scatterpolar(r=values, theta=categories + [categories[0]], fill='toself'))
        fig_radar.update_layout(polar=dict(radialaxis=dict(range=[0, 100])), title=f"Quality: {qs.overall:.0f}/100", height=400)
        figures_html += "<h3>Quality Score</h3>" + fig_radar.to_html(include_plotlyjs='cdn', full_html=False)
        
        progress.value = 0.6
        
        # 2. Timeseries
        timestamps = result.get_timestamps()
        if timestamps:
            ts = pd.to_datetime(timestamps)
            mean_cn0 = result.get_mean_cn0_series()
            sat_counts = result.get_satellite_count_series()
            
            fig_ts = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.7, 0.3])
            fig_ts.add_trace(go.Scatter(x=ts, y=mean_cn0, name='Mean CN0'), row=1, col=1)
            fig_ts.add_trace(go.Bar(x=ts, y=sat_counts, showlegend=False), row=2, col=1)
            fig_ts.update_layout(height=500, title='CN0 Timeseries')
            figures_html += "<h3>CN0 Timeseries</h3>" + fig_ts.to_html(include_plotlyjs=False, full_html=False)
        
        progress.value = 0.7
        
        # 3. Skyplot
        skyplot_data = result.get_skyplot_data()
        coverage = result.skyplot_coverage
        if skyplot_data:
            fig_sky = go.Figure()
            const_data = {}
            
            for sat_trace in skyplot_data:
                system = sat_trace.get('system', 'Other')
                const_name = {'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 'C': 'BeiDou'}.get(system, system)
                
                if const_name not in const_data:
                    const_data[const_name] = {'r': [], 'theta': [], 'cn0': []}
                
                azimuths = [float(x) for x in sat_trace.get('azimuths', '').split(',') if x]
                elevations = [float(x) for x in sat_trace.get('elevations', '').split(',') if x]
                cn0_values = [float(x) for x in sat_trace.get('cn0_values', '').split(',') if x]
                
                for az, el, cn0 in zip(azimuths, elevations, cn0_values):
                    const_data[const_name]['r'].append(90 - el)
                    const_data[const_name]['theta'].append(az)
                    const_data[const_name]['cn0'].append(cn0)
            
            for const_name, data in const_data.items():
                if data['r']:
                    fig_sky.add_trace(go.Scatterpolar(
                        r=data['r'], theta=data['theta'], mode='markers', name=const_name,
                        marker=dict(size=6, color=data['cn0'], colorscale='Viridis', cmin=25, cmax=55)
                    ))
            
            fig_sky.update_layout(title=f'Skyplot ({coverage:.1f}% coverage)', height=500, 
                                  polar=dict(radialaxis=dict(range=[0, 90])))
            figures_html += "<h3>Skyplot</h3>" + fig_sky.to_html(include_plotlyjs=False, full_html=False)
        
        progress.value = 0.75
        
        # 4. Time vs Satellite Heatmap
        if skyplot_data and timestamps:
            sat_data = {}
            for sat_trace in skyplot_data:
                sat_id = sat_trace.get('satellite', '')
                if not sat_id:
                    continue
                cn0_str = sat_trace.get('cn0_values', '')
                ts_str = sat_trace.get('timestamps', '')
                if cn0_str and ts_str:
                    cn0_vals = [float(x) for x in cn0_str.split(',') if x]
                    sat_ts = [x.strip() for x in ts_str.split(',') if x.strip()]
                    if len(cn0_vals) == len(sat_ts):
                        sat_data[sat_id] = dict(zip(sat_ts, cn0_vals))
            
            if sat_data:
                def sat_sort_key(s):
                    if len(s) >= 2:
                        sys = s[0]
                        try:
                            prn = int(s[1:])
                        except:
                            prn = 0
                        sys_order = {'G': 0, 'R': 1, 'E': 2, 'C': 3}
                        return (sys_order.get(sys, 9), prn)
                    return (9, 0)
                
                sorted_sats = sorted(sat_data.keys(), key=sat_sort_key, reverse=True)
                ts_list = pd.to_datetime(timestamps)
                z_matrix = []
                y_labels = []
                
                for sat_id in sorted_sats:
                    row = []
                    ts_cn0_map = sat_data[sat_id]
                    for t in timestamps:
                        cn0 = ts_cn0_map.get(t, None)
                        row.append(cn0)
                    z_matrix.append(row)
                    y_labels.append(sat_id)
                
                fig_time_sat = go.Figure(go.Heatmap(
                    z=z_matrix, x=ts_list, y=y_labels,
                    colorscale='Viridis', hoverongaps=False, zmin=25, zmax=55
                ))
                fig_time_sat.update_layout(title='C/N₀ Heatmap - Time vs Satellite', 
                                           height=max(400, len(sorted_sats) * 18 + 100))
                figures_html += "<h3>Time vs Satellite Heatmap</h3>" + fig_time_sat.to_html(include_plotlyjs=False, full_html=False)
        
        progress.value = 0.8
        
        # 5. Az/El Heatmap
        if skyplot_data:
            az_bins = list(range(0, 361, 15))
            el_bins = list(range(0, 91, 5))
            cn0_sum = [[0.0 for _ in range(len(az_bins)-1)] for _ in range(len(el_bins)-1)]
            cn0_count = [[0 for _ in range(len(az_bins)-1)] for _ in range(len(el_bins)-1)]
            
            for sat_trace in skyplot_data:
                azimuths = [float(x) for x in sat_trace.get('azimuths', '').split(',') if x]
                elevations = [float(x) for x in sat_trace.get('elevations', '').split(',') if x]
                cn0_values = [float(x) for x in sat_trace.get('cn0_values', '').split(',') if x]
                
                for az, el, cn0 in zip(azimuths, elevations, cn0_values):
                    az_idx = min(int(az / 15), len(az_bins) - 2)
                    el_idx = min(int(el / 5), len(el_bins) - 2)
                    cn0_sum[el_idx][az_idx] += cn0
                    cn0_count[el_idx][az_idx] += 1
            
            cn0_grid = []
            for el_idx in range(len(el_bins) - 1):
                row = []
                for az_idx in range(len(az_bins) - 1):
                    if cn0_count[el_idx][az_idx] > 0:
                        row.append(cn0_sum[el_idx][az_idx] / cn0_count[el_idx][az_idx])
                    else:
                        row.append(None)
                cn0_grid.append(row)
            
            if not all(all(v is None for v in row) for row in cn0_grid):
                fig_hm = go.Figure(go.Heatmap(z=cn0_grid, colorscale='Viridis', hoverongaps=False))
                fig_hm.update_layout(title='CN0 Heatmap by Az/El', height=400, xaxis_title='Azimuth', yaxis_title='Elevation')
                figures_html += "<h3>CN0 Heatmap (Az/El)</h3>" + fig_hm.to_html(include_plotlyjs=False, full_html=False)
        
        progress.value = 0.9
        
        # Build full HTML
        html = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>GeoVeil CN0 Report - {result.filename}</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; background: #f5f5f5; }}
        .container {{ max-width: 1200px; margin: 0 auto; background: white; padding: 30px; border-radius: 10px; }}
        h1 {{ color: #1a365d; border-bottom: 3px solid #3182ce; padding-bottom: 10px; }}
        h2 {{ color: #2c5282; }}
        .score {{ font-size: 48px; font-weight: bold; color: {'#22c55e' if qs.overall >= 80 else '#eab308' if qs.overall >= 60 else '#ef4444'}; }}
        .metric {{ display: inline-block; margin: 10px; padding: 15px; background: #f0f0f0; border-radius: 8px; }}
        .metric-value {{ font-size: 24px; font-weight: bold; }}
        .ok {{ color: #22c55e; }}
        .warn {{ color: #eab308; }}
        .danger {{ color: #ef4444; }}
    </style>
</head>
<body>
<div class="container">
    <h1>📡 GeoVeil CN0 Analysis Report</h1>
    
    <h2>📁 File Information</h2>
    <p><b>File:</b> {result.filename}<br>
    <b>Duration:</b> {result.duration_hours:.2f} hours ({result.epoch_count} epochs)<br>
    <b>Constellations:</b> {', '.join(result.get_systems())}</p>
    
    <h2>🏆 Quality Score</h2>
    <p class="score">{qs.overall:.0f}/100 ({qs.rating})</p>
    <div class="metric"><div class="metric-value">{qs.cn0_quality:.0f}</div>CN0 Quality</div>
    <div class="metric"><div class="metric-value">{qs.availability:.0f}</div>Availability</div>
    <div class="metric"><div class="metric-value">{qs.continuity:.0f}</div>Continuity</div>
    <div class="metric"><div class="metric-value">{qs.stability:.0f}</div>Stability</div>
    
    <h2>📶 Signal Quality</h2>
    <p><b>Mean CN0:</b> {result.mean_cn0:.1f} dB-Hz<br>
    <b>Std Dev:</b> {result.cn0_std_dev:.1f} dB-Hz<br>
    <b>Range:</b> {result.min_cn0:.1f} - {result.max_cn0:.1f} dB-Hz</p>
    
    <h2>🛡️ Threat Assessment</h2>
    <p><span class="{'danger' if result.jamming_detected else 'ok'}">Jamming: {'🚨 DETECTED' if result.jamming_detected else '✅ None'}</span><br>
    <span class="{'danger' if result.spoofing_detected else 'ok'}">Spoofing: {'🚨 DETECTED' if result.spoofing_detected else '✅ None'}</span><br>
    <span class="{'warn' if result.interference_detected else 'ok'}">Interference: {'⚠️ Detected' if result.interference_detected else '✅ None'}</span></p>
    
    <p><b>Summary:</b> {result.summary}</p>
    
    <h2>📊 Analysis Charts</h2>
    {figures_html}
    
    <h2>⚠️ Anomalies ({len(anomalies)} detected)</h2>
    {'<p>No significant anomalies detected.</p>' if not anomalies else f'<p>{len(anomalies)} anomaly events recorded. See anomaly chart above.</p>'}
    
    <hr>
    <p><small>Generated by GeoVeil CN0 v{gcn0.VERSION} on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</small></p>
</div>
</body>
</html>"""
        
        progress.value = 1.0
        
        # Save to temp file and offer download
        import tempfile
        import base64
        
        filename = f"cn0_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
        
        # Create download link
        b64 = base64.b64encode(html.encode()).decode()
        
        with results_out:
            clear_output()
            print(f"✅ Report generated: {filename}")
            print(f"   Quality Score: {qs.overall:.0f}/100 ({qs.rating})")
            print(f"   Anomalies: {len(anomalies)}")
            print()
            
            # Display download link
            download_link = f'<a download="{filename}" href="data:text/html;base64,{b64}" style="font-size:18px; padding:10px 20px; background:#22c55e; color:white; text-decoration:none; border-radius:5px;">📥 Download Report ({len(html)//1024} KB)</a>'
            display(HTML(download_link))
        
        progress.layout.visibility = 'hidden'
        
    except Exception as e:
        with results_out:
            clear_output()
            print(f"❌ Export error: {e}")
        progress.layout.visibility = 'hidden'


def clear_all(btn):
    """Clear all outputs"""
    with info_out:
        clear_output()
    with results_out:
        clear_output()
    analysis_results['data'] = None
    analysis_results['figures'] = {}
    status.value = "<b>Status:</b> Cleared"


def load_files(btn):
    """Load files from paths OR upload widgets"""
    with info_out:
        clear_output()
        print("📥 Loading files...")
        
        obs_loaded = False
        nav_loaded = False
        
        # ===== OBSERVATION FILE =====
        if obs_path_input.value.strip():
            path = obs_path_input.value.strip()
            print(f"\n📂 Loading OBS from path: {path}")
            
            if os.path.exists(path):
                try:
                    with open(path, 'rb') as f:
                        content = f.read()
                    
                    # Handle gzip
                    if path.endswith('.gz'):
                        content = gzip.decompress(content)
                    
                    loaded_data['obs_content'] = content
                    loaded_data['obs_filename'] = os.path.basename(path)
                    loaded_data['obs_path'] = path
                    obs_loaded = True
                    print(f"   ✅ Loaded: {loaded_data['obs_filename']} ({len(content)/1024:.1f} KB)")
                except Exception as e:
                    print(f"   ❌ Error: {e}")
            else:
                print(f"   ❌ File not found: {path}")
        
        elif obs_upload.value:
            file_info = list(obs_upload.value.values())[0]
            content = file_info['content']
            filename = file_info['metadata']['name']
            
            print(f"\n📤 Loading uploaded OBS: {filename}")
            
            if filename.endswith('.gz'):
                content = gzip.decompress(content)
                filename = filename[:-3]
            
            loaded_data['obs_content'] = content
            loaded_data['obs_filename'] = filename
            obs_loaded = True
            print(f"   ✅ Loaded: {filename} ({len(content)/1024:.1f} KB)")
        
        # ===== NAVIGATION FILE =====
        if nav_path_input.value.strip():
            path = nav_path_input.value.strip()
            print(f"\n📂 Loading NAV from path: {path}")
            
            if os.path.exists(path):
                try:
                    with open(path, 'rb') as f:
                        content = f.read()
                    
                    if path.endswith('.gz'):
                        content = gzip.decompress(content)
                    
                    loaded_data['nav_content'] = content
                    loaded_data['nav_filename'] = os.path.basename(path)
                    loaded_data['nav_path'] = path
                    nav_loaded = True
                    print(f"   ✅ Loaded: {loaded_data['nav_filename']} ({len(content)/1024:.1f} KB)")
                except Exception as e:
                    print(f"   ❌ Error: {e}")
            else:
                print(f"   ❌ File not found: {path}")
        
        elif nav_upload.value:
            file_info = list(nav_upload.value.values())[0]
            content = file_info['content']
            filename = file_info['metadata']['name']
            
            print(f"\n📤 Loading uploaded NAV: {filename}")
            
            if filename.endswith('.gz'):
                content = gzip.decompress(content)
                filename = filename[:-3]
            
            loaded_data['nav_content'] = content
            loaded_data['nav_filename'] = filename
            nav_loaded = True
            print(f"   ✅ Loaded: {filename} ({len(content)/1024:.1f} KB)")
        
        # ===== AUTO-DOWNLOAD NAV =====
        if not nav_loaded and obs_loaded and auto_download_nav.value:
            print("\n🌐 Attempting to auto-download navigation...")
            
            # Try to get date from file header first (most reliable)
            year, doy = NavDownloader.parse_rinex_header(loaded_data['obs_content'])
            
            # Fall back to filename parsing if header didn't work
            if not year or not doy:
                year, doy = NavDownloader.parse_rinex_date(loaded_data['obs_filename'])
            
            if year and doy:
                print(f"   Detected date: {year} DOY {doy}")
                
                nav_path = NavDownloader.download(year, doy, tempfile.gettempdir(), log_func=print)
                
                if nav_path and nav_path.exists():
                    with open(nav_path, 'rb') as f:
                        loaded_data['nav_content'] = f.read()
                    loaded_data['nav_filename'] = nav_path.name
                    loaded_data['nav_path'] = str(nav_path)
                    nav_loaded = True
            else:
                print("   ⚠️ Could not determine date from filename or header")
        
        # ===== SUMMARY =====
        print("\n" + "=" * 50)
        if obs_loaded:
            print(f"✅ OBS: {loaded_data['obs_filename']}")
        else:
            print("❌ OBS: Not loaded")
        
        if nav_loaded:
            print(f"✅ NAV: {loaded_data['nav_filename']}")
        else:
            print("⚠️ NAV: Not loaded (skyplots will be limited)")
        
        if obs_loaded:
            status.value = f"<b>Status:</b> ✅ Files loaded - click output buttons to analyze"
            analysis_results['data'] = None  # Clear old results
        else:
            status.value = "<b>Status:</b> ❌ No observation file loaded"


# ============ CONNECT HANDLERS ============
load_btn.on_click(load_files)
btn_summary.on_click(show_summary)
btn_heatmap.on_click(show_heatmap)
btn_snr.on_click(show_snr_graphs)
btn_skyplot.on_click(show_skyplot)
btn_anomaly.on_click(show_anomalies)
export_btn.on_click(export_report)
clear_btn.on_click(clear_all)

# ============ LAYOUT ============
file_box = widgets.VBox([
    header,
    obs_section,
    widgets.HBox([obs_upload, obs_path_input]),
    nav_section,
    widgets.HBox([nav_upload, nav_path_input]),
    widgets.HBox([auto_download_nav, load_btn]),
])

config_box = widgets.VBox([
    config_section,
    elevation_slider,
    time_bin_slider,
    widgets.HBox([system_checks['G'], system_checks['R'], system_checks['E'], system_checks['C']]),
])

# NEW: Output buttons in a row
output_buttons = widgets.HBox([
    btn_summary, btn_heatmap, btn_snr, btn_skyplot, btn_anomaly
], layout=widgets.Layout(margin='10px 0'))

action_box = widgets.VBox([
    buttons_section,
    output_buttons,
    widgets.HBox([export_btn, clear_btn]),
    progress,
    status
])

main_widget = widgets.VBox([
    file_box,
    widgets.HTML("<hr>"),
    config_box,
    widgets.HTML("<hr>"),
    action_box,
    widgets.HTML("<hr>"),
    info_out,
    results_out
])

# Display the widget
display(main_widget)
print("✅ Widget loaded - use buttons above to analyze GNSS data")

✅ Widget loaded - use buttons above to analyze GNSS data


In [ ]:
# Debug: Check what data structure we have
result = analysis_results.get('data')
if result:
    print("=== Available methods ===")
    methods = [m for m in dir(result) if not m.startswith('_')]
    for m in methods:
        print(f"  {m}")
    
    print("\n=== Skyplot data sample ===")
    skyplot = result.get_skyplot_data()
    if skyplot and len(skyplot) > 0:
        print(f"Number of satellite traces: {len(skyplot)}")
        print(f"First trace keys: {skyplot[0].keys() if isinstance(skyplot[0], dict) else type(skyplot[0])}")
        print(f"First trace: {skyplot[0]}")

In [ ]:
# Debug: Verify RINEX parsing and satellite counts
# Run this cell after loading files to compare direct parsing vs analysis

def debug_rinex_parsing():
    """Compare direct RINEX satellite count with analysis results"""
    import geoveil_cn0 as gcn0
    
    if not loaded_data.get('obs_content'):
        print("❌ No observation file loaded!")
        return
    
    # Parse RINEX directly
    # Try new name first, fall back to old
    try:
        obs = gcn0.parse_rinex(loaded_data['obs_content'], loaded_data['obs_filename'])
    except AttributeError:
        obs = gcn0.read_rinex_obs_bytes(loaded_data['obs_content'], loaded_data['obs_filename'])
    
    print("=" * 60)
    print("🔍 RINEX PARSING DEBUG INFO")
    print("=" * 60)
    
    # Get debug info
    debug = obs.debug_info()
    
    print(f"\n📄 File: {loaded_data['obs_filename']}")
    print(f"   Version: {debug.get('version', 'N/A')}")
    print(f"   Epochs: {debug.get('num_epochs', 'N/A')}")
    print(f"   Total Satellites: {debug.get('num_satellites', 'N/A')}")
    
    print(f"\n📡 SATELLITES BY CONSTELLATION:")
    sats_by_sys = obs.satellites_by_system()
    for sys in sorted(sats_by_sys.keys()):
        sats = sats_by_sys[sys]
        sys_name = {'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 'C': 'BeiDou', 'J': 'QZSS', 'I': 'NavIC'}.get(sys, sys)
        print(f"   {sys_name:10} ({sys}): {len(sats):2} satellites - {', '.join(sorted(sats))}")
    
    print(f"\n📊 OBSERVATION TYPES BY SYSTEM:")
    obs_types = obs.obs_types_by_system()
    for sys in sorted(obs_types.keys()):
        types = obs_types[sys]
        snr_types = [t for t in types if t.startswith('S')]
        sys_name = {'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 'C': 'BeiDou', 'J': 'QZSS', 'I': 'NavIC'}.get(sys, sys)
        print(f"   {sys_name:10} ({sys}): {len(types):2} types, {len(snr_types)} SNR")
        print(f"      All: {', '.join(types[:15])}{'...' if len(types) > 15 else ''}")
        if snr_types:
            print(f"      SNR: {', '.join(snr_types)}")
        else:
            print(f"      ⚠️  NO SNR TYPES FOUND!")
    
    print(f"\n🔬 SNR AVAILABILITY:")
    snr_avail = obs.snr_availability()
    for sys in sorted(snr_avail.keys()):
        has_snr = snr_avail[sys]
        sys_name = {'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 'C': 'BeiDou', 'J': 'QZSS', 'I': 'NavIC'}.get(sys, sys)
        status = "✅ Yes" if has_snr else "❌ No"
        print(f"   {sys_name:10} ({sys}): {status}")
    
    print("\n" + "=" * 60)
    return obs

# Run debug
if loaded_data.get('obs_content'):
    _debug_obs = debug_rinex_parsing()
else:
    print("Load files first, then run this cell")

def debug_visibility():
    """Debug visibility prediction"""
    if not loaded_data.get('nav_content'):
        print("⚠️ No navigation data loaded - visibility prediction unavailable")
        return
    
    result = analysis_results.get('data') if 'analysis_results' in dir() else None
    if not result or not result.has_visibility_prediction:
        print("⚠️ No visibility prediction available")
        return
    
    print("=" * 60)
    print("🔍 VISIBILITY PREDICTION DEBUG")
    print("=" * 60)
    
    vis_debug = result.visibility_debug()
    
    # Overall stats
    if '_overall' in vis_debug:
        overall = vis_debug['_overall']
        print(f"\n📡 Source: {overall.get('source', 'unknown')}")
        print(f"   Mean predicted: {overall.get('mean_predicted', 'N/A')} satellites/epoch")
        print(f"   Mean observed: {overall.get('mean_observed', 'N/A')} satellites/epoch")
        print(f"   Mean missing: {overall.get('mean_missing', 'N/A')}")
        print(f"   Mean unexpected: {overall.get('mean_unexpected', 'N/A')}")
        print(f"   Confirmation rate: {overall.get('confirmation_rate', 'N/A')}")
    
    print("\n🛰️ PER-SYSTEM BREAKDOWN:")
    for sys_name in ['GPS', 'GLONASS', 'Galileo', 'BeiDou']:
        if sys_name in vis_debug:
            info = vis_debug[sys_name]
            predicted = info.get('predicted', '0')
            observed = info.get('observed', '0')
            conf_rate = info.get('confirmation_rate', 'N/A')
            unexpected = info.get('unexpected_count', '0')
            unexpected_sats = info.get('unexpected_sats', '')
            
            print(f"\n   {sys_name}:")
            print(f"      Predicted (from nav): {predicted}")
            print(f"      Observed (in RINEX): {observed}")
            print(f"      Confirmation rate: {conf_rate}")
            if int(unexpected) > 0:
                print(f"      ⚠️ Unexpected satellites: {unexpected} ({unexpected_sats})")
                print(f"         (These are in RINEX but not in nav file)")

# Run if navigation data available
if loaded_data.get('nav_content') and 'analysis_results' in dir() and analysis_results.get('data'):
    debug_visibility()

def debug_satellite_comparison():
    """Compare satellites in RINEX vs Analysis result"""
    if not loaded_data.get('obs_content'):
        print("Load files first")
        return
    
    result = analysis_results.get('data') if 'analysis_results' in dir() else None
    if not result:
        print("Run analysis first")
        return
    
    import geoveil_cn0 as gcn0
    import json as js
    
    # Parse RINEX
    try:
        obs = gcn0.parse_rinex(loaded_data['obs_content'], loaded_data['obs_filename'])
    except AttributeError:
        obs = gcn0.read_rinex_obs_bytes(loaded_data['obs_content'], loaded_data['obs_filename'])
    
    rinex_sats = obs.satellites_by_system()
    
    # Get analysis result satellites from JSON
    result_json = js.loads(result.to_json())
    timeseries_sats = result_json.get('timeseries', {}).get('satellite_timeseries', {}).keys()
    
    print("=" * 60)
    print("🔍 SATELLITE COMPARISON: RINEX vs Analysis")
    print("=" * 60)
    
    sys_names = {'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 'C': 'BeiDou', 'J': 'QZSS', 'I': 'NavIC'}
    
    for sys_code in sorted(rinex_sats.keys()):
        sys_name = sys_names.get(sys_code, sys_code)
        rinex_set = set(rinex_sats[sys_code])
        ts_set = set(s for s in timeseries_sats if s.startswith(sys_code))
        
        in_rinex_only = rinex_set - ts_set
        in_ts_only = ts_set - rinex_set
        in_both = rinex_set & ts_set
        
        print(f"\n{sys_name} ({sys_code}):")
        print(f"   RINEX: {len(rinex_set)} satellites - {', '.join(sorted(rinex_set))}")
        print(f"   Timeseries: {len(ts_set)} satellites - {', '.join(sorted(ts_set))}")
        
        if in_rinex_only:
            print(f"   ⚠️  Missing from timeseries: {', '.join(sorted(in_rinex_only))}")
            print(f"      (These satellites have no valid SNR data or are filtered out)")
        if in_ts_only:
            print(f"   ❓ Extra in timeseries: {', '.join(sorted(in_ts_only))}")
    
    print("\n" + "=" * 60)

# Run if both data available
if loaded_data.get('obs_content') and 'analysis_results' in dir() and analysis_results.get('data'):
    debug_satellite_comparison()

def debug_snr_content():
    """Analyze SNR data availability in RINEX file"""
    if not loaded_data.get('obs_content'):
        print("Load files first")
        return
    
    import geoveil_cn0 as gcn0
    
    try:
        obs = gcn0.parse_rinex(loaded_data['obs_content'], loaded_data['obs_filename'])
    except AttributeError:
        obs = gcn0.read_rinex_obs_bytes(loaded_data['obs_content'], loaded_data['obs_filename'])
    
    print("=" * 60)
    print("🔬 SNR DATA ANALYSIS PER SATELLITE")
    print("=" * 60)
    
    # Get SNR statistics per satellite
    snr_stats = obs.snr_stats_by_satellite()
    
    sys_names = {'G': 'GPS', 'R': 'GLONASS', 'E': 'Galileo', 'C': 'BeiDou', 'J': 'QZSS', 'I': 'NavIC'}
    
    # Group by system
    by_system = {}
    for sat_id, stats in snr_stats.items():
        sys_code = sat_id[0]
        if sys_code not in by_system:
            by_system[sys_code] = []
        by_system[sys_code].append((sat_id, stats))
    
    for sys_code in sorted(by_system.keys()):
        sys_name = sys_names.get(sys_code, sys_code)
        print(f"\n{sys_name} ({sys_code}):")
        
        for sat_id, stats in sorted(by_system[sys_code]):
            obs_count = stats.get('obs_count', 0)
            valid_count = stats.get('valid_count', 0)
            mean_snr = stats.get('mean_snr', 0)
            
            status = "✅" if valid_count > 0 else "❌"
            print(f"   {sat_id}: {status} {valid_count}/{obs_count} valid SNR obs, mean={mean_snr:.1f} dB-Hz")
    
    print("\n" + "=" * 60)
    print("Satellites with ❌ have no valid SNR data and won't appear in timeseries")

# Uncomment to run:
# debug_snr_content()


In [ ]:
%%bash
# Extract and build library
[ ! -d geoveil-cn0 ] && [ -f geoveil-cn0.tar.gz ] && tar -xzf geoveil-cn0.tar.gz

# Source cargo environment (required after fresh install)
. "$HOME/.cargo/env"

cd geoveil-cn0
echo "🔨 Building with Python bindings..."
maturin develop --features python --release 2>&1 | tail -15
echo "✅ Build complete!"

##### 5. Programmatic API

In [ ]:
# Quick: result = gcn0.analyze_rinex('file.rnx')
# Config: config = gcn0.AnalysisConfig(min_elevation=10, anomaly_sensitivity=0.8)
#         result = gcn0.CN0Analyzer(config).analyze_file('file.rnx')
# Presets: gcn0.AnalysisConfig.high_precision() / .quick() / .interference_monitoring()
# Results: result.score, result.rating, result.jamming_detected, result.get_anomalies()
#          result.get_timeseries_data(), result.get_skyplot_data(), result.to_json()

## API Reference

| Class | Description |
|-------|-------------|
| `CN0Analyzer` | Rust analyzer |
| `AnalysisConfig` | Config + presets |
| `AnalysisResult` | Results container |

**90% Rust** - parsing, analysis, anomaly detection. **10% Python** - UI & plots.